In [1]:
import math 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib notebook
from matplotlib import gridspec

import uproot3 as uproot

# Plots setting
plt.rcParams['axes.labelsize'] = 15
tick_size = 13
plt.rcParams['legend.frameon'] = False
plt.rcParams['legend.title_fontsize'] = 13
legend_size = 13

In [2]:
from scipy.stats import norm, chisquare 
from scipy.optimize import curve_fit, leastsq
from scipy.special import logit

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, roc_auc_score, accuracy_score, f1_score

import xgboost as xgb
from xgboost import XGBClassifier

### Import functions.py

In [3]:
# FUNCTIONS FOR EVENT SELECTION
# ----------------------------------------------------------------- EVENT SELECTION
def is_truth_nueCC(df):
    df_ = df[((abs(df.truth_nuPdg) == 12) & (df.truth_isCC == 1))]
    return df_

def is_truth_nueCC_inside(df):
    df_ = df[((abs(df.truth_nuPdg) == 12) & (df.truth_isCC == 1) & (df.truth_vtxInside == 1))]
    return df_

def is_truth_nueCC_outside(df):
    df_ = df[((abs(df.truth_nuPdg) == 12) & (df.truth_isCC == 1) & (df.truth_vtxInside == 0))]
    return df_

def is_truth_numuCC(df):
    df_ = df[((abs(df.truth_nuPdg) == 14) & (df.truth_isCC == 1))]
    return df_

def is_truth_numuCC_inside(df):
    df_ = df[((abs(df.truth_nuPdg) == 14) & (df.truth_isCC == 1) & (df.truth_vtxInside == 1))]
    return df_

def is_truth_numuCC_outside(df):
    df_ = df[((abs(df.truth_nuPdg) == 14) & (df.truth_isCC == 1) & (df.truth_vtxInside == 0))]
    return df_

def is_truth_NC(df):
    df_ = df[(df.truth_isCC == 0)]
    return df_

def is_truth_NC_inside(df):
    df_ = df[((df.truth_isCC == 0) & (df.truth_vtxInside == 1))]
    return df_

def is_truth_NC_outside(df):
    df_ = df[((df.truth_isCC == 0) & (df.truth_vtxInside == 0))]
    return df_

def is_FC(df):
    df_ = df[df.match_isFC == 1]
    return df_

def is_PC(df):
    df_ = df[df.match_isFC == 0]
    return df_

em_charge_scale = 0.95

def is_pi0(df, family='MC'):
    if family == 'DATA': df.loc[:,'kine_pio_mass'] = df.kine_pio_mass * em_charge_scale
    elif family == 'MC': pass
    else: print(' -------- WARNING: Wrong family')
        
    df_ = df[((df.kine_pio_flag == 1) & (df.kine_pio_vtx_dis < 9) | (df.kine_pio_flag == 2)) &
             (df.kine_pio_energy_1 > 40) & 
             (df.kine_pio_energy_2 > 25) & 
             (df.kine_pio_dis_1 < 110) & 
             (df.kine_pio_dis_2 < 120) & 
             (df.kine_pio_angle > 0) & 
             (df.kine_pio_angle < 174) & 
             (df.kine_pio_mass > 22) & 
             (df.kine_pio_mass < 300)]
    return df_

def is_NC(df):
    df_ = df[((df.cosmict_flag == 0) & (df.numu_score < 0))]
    return df_

def is_numuCC(df):
    df_ = df[((df.numu_cc_flag >= 0) & (df.numu_score > 0.9))]
    return df_

def is_nueCC(df):
    df_ = df[((df.numu_cc_flag >= 0) & (df.nue_score > 7))]
    return df_

def is_0p(df):
    df_ = df[df.Num_protons == 0]
    return df_

def is_Np(df):
    df_ = df[df.Num_protons > 0]
    return df_

def is_0pi(df):
    df_ = df[df.Num_pions == 0]
    return df_

def is_Npi(df):
    df_ = df[df.Num_pions > 0]
    return df_

def is_generic(df):
    df_ = df[(df.match_found == 1) & 
             (df.stm_eventtype != 0) &
             (df.stm_lowenergy == 0) &
             (df.stm_LM == 0) &
             (df.stm_TGM == 0) &
             (df.stm_STM == 0) &
             (df.stm_FullDead == 0) &
             (df.stm_clusterlength > 15)]
    return df_

def isnot_generic(df):
    df_ = df[~((df.match_found == 1) & 
               (df.stm_eventtype != 0) &
               (df.stm_lowenergy == 0) &
               (df.stm_LM == 0) &
               (df.stm_TGM == 0) &
               (df.stm_STM == 0) &
               (df.stm_FullDead == 0) &
               (df.stm_clusterlength > 15))]
    return df_

def is_preselection(df):
    df_ = df[(df.match_found == 1) & 
             (df.stm_eventtype != 0) &
             (df.stm_lowenergy == 0) &
             (df.stm_LM == 0) &
             (df.stm_TGM == 0) &
             (df.stm_STM == 0) &
             (df.stm_FullDead == 0) &
             (df.stm_clusterlength > 0)]
    return df_

def Cosmic(df):
    # Badreco match or events with no true energy inside
    df_ = df[((df.match_completeness_energy <= df.truth_energyInside * 0.1) | (df.truth_energyInside <= 0))]
    return df_

def not_Cosmic(df):
    # Opposite of Cosmic
    df_ = df[~((df.match_completeness_energy <= df.truth_energyInside * 0.1) | (df.truth_energyInside <= 0))]
    return df_

def Active(df):
    # Active volume (truth)
    df_ = df[(df.truth_vtxX > -1) &
             (df.truth_vtxX <= 254.3) &
             (df.truth_vtxY > -115.0) &
             (df.truth_vtxY <= 117.0) &
             (df.truth_vtxZ > 0.6) &
             (df.truth_vtxZ <= 1036.4)]
    return df_


# FUNCTIONS FOR EVENT PRE-PROCESSING
# --------------------------------------------------------------------------------------- PRE-PROCESSING
def gen_run_subrun_list(input_file, name_list):
    # Get useful variables
    T_eval = uproot.open(input_file)['wcpselection/T_eval']
    df_eval = T_eval.pandas.df(['run', 'subrun'], flatten=False)
    df_eval.drop_duplicates(inplace=True) 
    np.savetxt(name_list, df_eval.values, fmt='%d')

BDT_variab_all = [#'cosmic_n_solid_tracks', 'cosmic_energy_main_showers',
                  #'cosmic_energy_direct_showers', 'cosmic_energy_indirect_showers',
                  #'cosmic_n_direct_showers', 'cosmic_n_indirect_showers',
                  #'cosmic_n_main_showers', 
                  # -----------------------------------------------
                  #'gap_flag_prolong_u', 'gap_flag_prolong_v',
                  #'gap_flag_prolong_w', 'gap_flag_parallel',
                  #'gap_n_points', 'gap_n_bad',
                  #'gap_energy', 'gap_num_valid_tracks',
                  #'gap_flag_single_shower', 
                  # -----------------------------------------------
                  #'mip_quality_energy', 'mip_quality_overlap',
                  #'mip_quality_n_showers', 'mip_quality_n_tracks', 
                  #'mip_quality_flag_inside_pi0', 'mip_quality_n_pi0_showers', 
                  #'mip_quality_shortest_length', 'mip_quality_acc_length',
                  #'mip_quality_shortest_angle', 'mip_quality_flag_proton',
                  # -----------------------------------------------
                  #'mip_energy',
                  #'mip_n_end_reduction', 'mip_n_first_mip',
                  #'mip_n_first_non_mip', 'mip_n_first_non_mip_1',
                  #'mip_n_first_non_mip_2', 'mip_vec_dQ_dx_0',
                  #'mip_vec_dQ_dx_1', 'mip_vec_dQ_dx_2',
                  #'mip_vec_dQ_dx_3', 'mip_vec_dQ_dx_4',
                  #'mip_vec_dQ_dx_5', 'mip_vec_dQ_dx_6',
                  #'mip_vec_dQ_dx_7', 'mip_vec_dQ_dx_8',
                  #'mip_vec_dQ_dx_9', 'mip_vec_dQ_dx_10',
                  #'mip_vec_dQ_dx_11', 'mip_vec_dQ_dx_12',
                  #'mip_vec_dQ_dx_13', 'mip_vec_dQ_dx_14',
                  #'mip_vec_dQ_dx_15', 'mip_vec_dQ_dx_16',
                  #'mip_vec_dQ_dx_17', 'mip_vec_dQ_dx_18',
                  #'mip_vec_dQ_dx_19', 'mip_max_dQ_dx_sample',
                  #'mip_n_below_threshold', 'mip_n_below_zero',
                  #'mip_n_lowest', 'mip_n_highest',
                  #'mip_lowest_dQ_dx', 'mip_highest_dQ_dx',
                  #'mip_medium_dQ_dx', 'mip_stem_length',
                  #'mip_length_main', 'mip_length_total',
                  #'mip_angle_beam', 'mip_iso_angle',
                  #'mip_n_vertex', 'mip_n_good_tracks',
                  #'mip_E_indirect_max_energy', 'mip_flag_all_above',
                  #'mip_min_dQ_dx_5', 'mip_n_other_vertex',
                  #'mip_n_stem_size', 'mip_flag_stem_trajectory',
                  #'mip_min_dis', 
                  # -----------------------------------------------
                  #'pio_mip_id', 'pio_flag_pio', 
                  # -----------------------------------------------
                  #'pio_1_mass', 'pio_1_pio_type', 
                  #'pio_1_energy_1', 'pio_1_energy_2', 
                  #'pio_1_dis_1', 'pio_1_dis_2', 
                  # -----------------------------------------------
                  #'mgo_energy',
                  #'mgo_max_energy', 'mgo_total_energy',
                  #'mgo_n_showers', 'mgo_max_energy_1',
                  #'mgo_max_energy_2', 'mgo_total_other_energy',
                  #'mgo_n_total_showers', 'mgo_total_other_energy_1',
                  # -----------------------------------------------
                  #'mgt_flag_single_shower', 'mgt_max_energy',
                  #'mgt_energy', 'mgt_total_other_energy',
                  #'mgt_max_energy_1', 'mgt_e_indirect_max_energy',
                  #'mgt_e_direct_max_energy', 'mgt_n_direct_showers',
                  #'mgt_e_direct_total_energy', 'mgt_flag_indirect_max_pio',
                  #'mgt_e_indirect_total_energy',
                  # -----------------------------------------------
                  #'stw_1_energy', 'stw_1_dis',
                  #'stw_1_dQ_dx', 'stw_1_flag_single_shower',
                  #'stw_1_n_pi0', 'stw_1_num_valid_tracks',
                  # -----------------------------------------------
                  #'spt_flag_single_shower', 'spt_energy',
                  #'spt_shower_main_length', 'spt_shower_total_length',
                  #'spt_angle_beam', 'spt_angle_vertical',
                  #'spt_max_dQ_dx', 'spt_angle_beam_1',
                  #'spt_angle_drift', 'spt_angle_drift_1',
                  #'spt_num_valid_tracks', 'spt_n_vtx_segs',
                  #'spt_max_length', 
                  # -----------------------------------------------
                  #'stem_len_energy', 'stem_len_length',
                  #'stem_len_flag_avoid_muon_check',
                  #'stem_len_num_daughters', 'stem_len_daughter_length',
                  # -----------------------------------------------
                  #'lem_shower_total_length',
                  #'lem_shower_main_length', 'lem_n_3seg',
                  #'lem_e_charge', 'lem_e_dQdx',
                  #'lem_shower_num_segs', 'lem_shower_num_main_segs',
                  # -----------------------------------------------
                  #'brm_n_mu_segs', 'brm_Ep',
                  #'brm_energy', 'brm_acc_length',
                  #'brm_shower_total_length', 'brm_connected_length',
                  #'brm_n_size', 'brm_acc_direct_length',
                  #'brm_n_shower_main_segs', 'brm_n_mu_main',
                  # -----------------------------------------------
                  #'cme_mu_energy', 'cme_energy',
                  #'cme_mu_length', 'cme_length',
                  #'cme_angle_beam',
                  # -----------------------------------------------
                  #'anc_energy', 'anc_angle',
                  #'anc_max_angle', 'anc_max_length',
                  #'anc_acc_forward_length', 'anc_acc_backward_length',
                  #'anc_acc_forward_length1', 'anc_shower_main_length',
                  #'anc_shower_total_length', 'anc_flag_main_outside',
                  # -----------------------------------------------
                  #'stem_dir_flag_single_shower', 'stem_dir_angle',
                  #'stem_dir_energy', 'stem_dir_angle1',
                  #'stem_dir_angle2', 'stem_dir_angle3',
                  #'stem_dir_ratio',
                  # -----------------------------------------------
                  #'vis_1_n_vtx_segs', 'vis_1_energy',
                  #'vis_1_num_good_tracks', 'vis_1_max_angle',
                  #'vis_1_max_shower_angle', 'vis_1_tmp_length1',
                  #'vis_1_tmp_length2', 'vis_1_particle_type',                                      
                  # -----------------------------------------------
                  #'vis_2_n_vtx_segs', 'vis_2_min_angle',
                  #'vis_2_min_weak_track', 'vis_2_angle_beam',
                  #'vis_2_min_angle1', 'vis_2_iso_angle1',
                  #'vis_2_min_medium_dQ_dx', 'vis_2_min_length',
                  #'vis_2_sg_length', 'vis_2_max_angle',
                  #'vis_2_max_weak_track',
                  # -----------------------------------------------
                  #'br1_1_shower_type',
                  #'br1_1_vtx_n_segs', 'br1_1_energy',
                  #'br1_1_n_segs', 'br1_1_flag_sg_topology',
                  #'br1_1_flag_sg_trajectory', 'br1_1_sg_length',
                  # -----------------------------------------------
                  #'br1_2_energy', 'br1_2_n_connected',
                  #'br1_2_max_length', 'br1_2_n_connected_1',
                  #'br1_2_vtx_n_segs', 'br1_2_n_shower_segs',
                  #'br1_2_max_length_ratio', 'br1_2_shower_length',
                  # -----------------------------------------------
                  #'br1_3_energy', 'br1_3_n_connected_p',
                  #'br1_3_max_length_p', 'br1_3_n_shower_segs',
                  #'br1_3_flag_sg_topology', 'br1_3_flag_sg_trajectory',
                  #'br1_3_n_shower_main_segs', 'br1_3_sg_length',
                  # -----------------------------------------------
                  #'br2_flag_single_shower', 'br2_num_valid_tracks',
                  #'br2_energy', 'br2_angle1',
                  #'br2_angle2', 'br2_angle',
                  #'br2_angle3', 'br2_n_shower_main_segs',
                  #'br2_max_angle', 'br2_sg_length',
                  #'br2_flag_sg_trajectory',                                     
                  # -----------------------------------------------
                  #'br3_1_n_shower_segments', 'br3_1_sg_flag_trajectory',
                  #'br3_1_sg_direct_length', 'br3_1_sg_length',
                  #'br3_1_total_main_length', 'br3_1_total_length',
                  #'br3_1_iso_angle', 'br3_1_sg_flag_topology',
                  # -----------------------------------------------
                  #'br3_2_n_ele', 'br3_2_n_other',
                  #'br3_2_energy', 'br3_2_total_main_length',
                  #'br3_2_total_length', 'br3_2_other_fid',
                  # -----------------------------------------------
                  #'br3_4_acc_length', 'br3_4_total_length',
                  #'br3_4_energy', 
                  # -----------------------------------------------
                  #'br3_7_energy', 'br3_7_min_angle', 
                  #'br3_7_sg_length', 'br3_7_main_length', 
                  # -----------------------------------------------
                  #'br3_8_max_dQ_dx', 'br3_8_energy', 'br3_8_n_main_segs',
                  #'br3_8_shower_main_length', 'br3_8_shower_length',
                  # -----------------------------------------------
                  #'br4_1_shower_main_length', 'br4_1_shower_total_length',
                  #'br4_1_min_dis', 'br4_1_energy',
                  #'br4_1_n_vtx_segs', 'br4_1_n_main_segs',
                  # -----------------------------------------------
                  #'br4_2_ratio_45', 'br4_2_ratio_35',
                  #'br4_2_ratio_25', 'br4_2_ratio_15',
                  #'br4_2_energy', 'br4_2_ratio1_45',
                  #'br4_2_ratio1_35', 'br4_2_ratio1_25',
                  #'br4_2_ratio1_15', 'br4_2_iso_angle',
                  #'br4_2_iso_angle1', 'br4_2_angle',                                     
                  # -----------------------------------------------
                  #'tro_3_stem_length', 'tro_3_n_muon_segs',
                  #'tro_3_energy',
                  # -----------------------------------------------
                  #'hol_1_n_valid_tracks', 'hol_1_min_angle',
                  #'hol_1_energy', 'hol_1_flag_all_shower',
                  #'hol_1_min_length', 
                  # -----------------------------------------------
                  #'hol_2_min_angle', 'hol_2_medium_dQ_dx',
                  #'hol_2_ncount', 'hol_2_energy',
                  # -----------------------------------------------
                  #'lol_3_angle_beam', 'lol_3_n_valid_tracks', 
                  #'lol_3_min_angle', 'lol_3_vtx_n_segs', 
                  #'lol_3_energy', 'lol_3_shower_main_length', 
                  #'lol_3_n_out', 'lol_3_n_sum',
                  # -----------------------------------------------
                  #'cosmict_2_particle_type', 'cosmict_2_n_muon_tracks',
                  #'cosmict_2_flag_inside', 'cosmict_2_angle_beam',
                  #'cosmict_2_flag_dir_weak', 'cosmict_2_dQ_dx_end',
                  #'cosmict_2_dQ_dx_front', 'cosmict_2_theta',
                  #'cosmict_2_phi', 'cosmict_2_valid_tracks',
                  # -----------------------------------------------
                  #'cosmict_3_flag_inside',
                  #'cosmict_3_angle_beam', 'cosmict_3_flag_dir_weak',
                  #'cosmict_3_dQ_dx_end', 'cosmict_3_dQ_dx_front',
                  #'cosmict_3_theta', 'cosmict_3_phi',
                  #'cosmict_3_valid_tracks', 
                  # -----------------------------------------------
                  #'cosmict_4_flag_inside', 'cosmict_4_angle_beam',
                  # -----------------------------------------------
                  #'cosmict_5_flag_inside', 'cosmict_5_angle_beam', 
                  # -----------------------------------------------
                  #'cosmict_6_flag_dir_weak', 'cosmict_6_flag_inside',
                  #'cosmict_6_angle', 
                  # -----------------------------------------------
                  #'cosmict_7_flag_sec', 'cosmict_7_n_muon_tracks',
                  #'cosmict_7_flag_inside', 'cosmict_7_angle_beam',
                  #'cosmict_7_flag_dir_weak', 'cosmict_7_dQ_dx_end',
                  #'cosmict_7_dQ_dx_front', 'cosmict_7_theta',
                  #'cosmict_7_phi', 
                  # -----------------------------------------------
                  #'cosmict_8_flag_out', 'cosmict_8_muon_length',
                  #'cosmict_8_acc_length',
                  # -----------------------------------------------
                  #'numu_cc_3_particle_type',
                  #'numu_cc_3_max_length', 'numu_cc_3_track_length',
                  #'numu_cc_3_max_length_all', 'numu_cc_3_max_muon_length',
                  #'numu_cc_3_n_daughter_all', 
                  # -----------------------------------------------                                      
                  #'pio_2_score', 'sig_1_score',
                  #'sig_2_score', 'stw_2_score',
                  #'stw_3_score', 'stw_4_score',
                  #'br3_3_score', 'br3_5_score',
                  #'br3_6_score', 'lol_1_score',
                  #'lol_2_score', 'tro_1_score',
                  #'tro_2_score', 'tro_4_score',
                  #'tro_5_score', 'cosmict_10_score',
                  #'numu_1_score', 'numu_2_score',
                  'numu_score', 'nue_score',
                  #'nc_delta_score','nc_pio_score', # These 2 are only added for fake data
                  'nc_pio_score',
                  'cosmict_flag', 'numu_cc_flag']  # Last 2 or 4 variables should not be included in training

KINE_variables = ['kine_reco_Enu', 'kine_reco_add_energy',
                    'kine_pio_mass', 'kine_pio_flag',
                    'kine_pio_vtx_dis', 'kine_pio_energy_1',
                    'kine_pio_theta_1', 'kine_pio_phi_1',
                    'kine_pio_dis_1', 'kine_pio_energy_2',
                    'kine_pio_theta_2', 'kine_pio_phi_2',
                    'kine_pio_dis_2', 'kine_pio_angle']
                    #'kine_energy_particle', 'kine_energy_info',
                    #'kine_particle_type', 'kine_energy_included'] # Last 4 variables are vectors and are excluded at the moment

eval_variables_MC = ['run', 'subrun', 'event',
                     #'truth_isCC', 'truth_nuPdg', 'truth_nuEnergy', 'truth_vtxInside', 
                     #'truth_vtxX', 'truth_vtxY', 'truth_vtxZ', 'weight_spline', 'weight_cv', 
                     'weight_lee', 'truth_energyInside', 'match_completeness_energy', 
                     'match_isFC', 'stm_clusterlength', 'match_found', 'stm_eventtype', 
                     'stm_lowenergy', 'stm_LM', 'stm_TGM', 'stm_STM', 'stm_FullDead']

eval_variables_DATA = ['run', 'subrun', 'event',
                       'match_isFC', 'stm_clusterlength', 'match_found', 'stm_eventtype', 
                       'stm_lowenergy', 'stm_LM', 'stm_TGM', 'stm_STM', 'stm_FullDead']

pfeval_variables_PF = ['truth_NprimPio', 'truth_NCDelta', 'truth_pio_energy_1', 'truth_pio_energy_2',
                       'truth_pio_angle', 'mc_nu_mode', 'truth_Ntrack', 'reco_Ntrack',
                       'reco_Nproton',  
                       'truth_pdg', 'truth_mother', 'truth_startMomentum']

pfeval_variables = ['truth_NprimPio', 'truth_NCDelta', 'truth_pio_energy_1', 'truth_pio_energy_2',
                    'truth_pio_angle']

Non_unique = ['mip_quality_energy', 'mgo_energy', 'mgt_energy', 'stw_1_energy', 'spt_energy', 
              'stem_len_energy', 'brm_energy', 'cme_energy', 'anc_energy', 'stem_dir_energy', 
              'br1_1_energy', 'br1_2_energy', 'br1_3_energy', 'br2_energy', 'br3_2_energy', 
              'br3_4_energy', 'br3_7_energy', 'br3_8_energy', 'br4_1_energy', 'br4_2_energy', 
              'tro_3_energy', 'lol_3_energy', 'br2_num_valid_tracks', 'mgt_flag_single_shower', 
              'stw_1_flag_single_shower', 'spt_flag_single_shower', 'stem_dir_flag_single_shower', 
              'br2_flag_single_shower', 'spt_angle_drift', 'mgt_max_energy', 'br1_1_flag_sg_trajectory', 
              'br1_3_flag_sg_trajectory', 'br2_flag_sg_trajectory', 'br3_1_sg_flag_trajectory', 
              'spt_max_dQ_dx', 'lem_shower_main_length', 'anc_shower_main_length', 
              'br3_1_total_main_length', 'br3_2_total_main_length', 'br3_4_total_length', 
              'br3_7_main_length', 'br3_8_shower_main_length', 'br4_1_shower_main_length', 
              'lol_3_shower_main_length', 'lem_shower_total_length', 'brm_shower_total_length', 
              'cme_length', 'anc_shower_total_length', 'br1_2_shower_length', 'br3_2_total_length', 
              'br3_8_shower_length', 'br4_1_shower_total_length', 'anc_angle', 'br1_1_vtx_n_segs', 
              'br1_2_vtx_n_segs', 'br4_1_n_vtx_segs', 'lol_3_vtx_n_segs', 'br1_1_sg_length', 
              'br1_3_sg_length', 'br2_sg_length', 'br3_1_sg_length', 'br3_7_sg_length', 
              'brm_n_shower_main_segs', 'br1_1_n_segs', 'br1_2_n_shower_segs', 'br1_3_n_shower_segs', 
              'br3_1_n_shower_segments', 'br1_3_n_shower_main_segs', 'br2_n_shower_main_segs', 
              'br3_8_n_main_segs', 'br4_1_n_main_segs', 'br2_angle', 'br2_angle1', 'br2_angle2', 
              'br2_angle3', 'vis_1_tmp_length2', 'br1_3_flag_sg_topology', 'br3_1_sg_flag_topology', 
              'hol_2_energy', 'cosmict_2_theta', 'cosmict_4_angle_beam', 'cosmict_5_flag_inside', 
              'cosmict_3_theta', 'cosmict_5_angle_beam', 'cosmict_7_theta']

BDT_variables = [x for x in BDT_variab_all if x not in Non_unique]
    
def gen_dataframe(input_file, tag, family='MC', POT_file=None):
    # tag: it's just a printed info
    # family: MC / DATA
    # POT_file: useful if family == DATA
    
    ############################################################ Import T_pot
    if family=='MC':
        # Calculate POT and scaling factor
        T_pot = uproot.open(input_file)['wcpselection/T_pot']
        df_pot = T_pot.pandas.df(T_pot.keys(), flatten=False)
        df_pot = df_pot.drop_duplicates(subset=['runNo','subRunNo'])
        if POT_file == None: POT_file = sum(df_pot.pot_tor875)
        elif POT_file != None: POT_file = POT_file
        print('\033[1m'+'%s [%s]: POT = %s'%(tag, family, POT_file)+'\033[0m')
    elif family=='DATA':
        # Calculate POT and scaling factor
        POT_file = POT_file
        print('\033[1m'+'%s [%s]: POT = %s'%(tag, family, POT_file)+'\033[0m')
    else: print(' -------- WARNING: Wrong family')
        
    ############################################################ Import T_KINEvars
    T_KINEvars = uproot.open(input_file)['wcpselection/T_KINEvars']
    df_KINEvars = T_KINEvars.pandas.df(KINE_variables, flatten=False)
    
    KINE_particles = ['kine_energy_particle','kine_particle_type']
    df_KINEpart = T_KINEvars.pandas.df(KINE_particles, flatten=False)
    
    # Count reco protons and pions w/ thresholds
    df_KINEpart.loc[:,'Num_protons'] = [0]*df_KINEpart.shape[0]
    df_KINEpart.loc[:,'Num_nonreco_protons'] = [0]*df_KINEpart.shape[0]
    df_KINEpart.loc[:,'Num_total_protons'] = [0]*df_KINEpart.shape[0]
    kine_reco_protons = []
    #df_KINEpart.loc[:,'Num_pions'] = [0]*df_KINEpart.shape[0]
    threshold_P = 35
    threshold_pi = 10
    for x,y,w in zip(df_KINEpart.kine_particle_type, df_KINEpart.kine_energy_particle, range(len(df_KINEpart.kine_particle_type))):
        if (len(x) > 0) & (2212 in x): 
            index_P = [i for i, z in enumerate(x) if z == 2212]
            df_KINEpart.loc[w,'Num_protons'] = sum([True if Y > threshold_P else False for Y in y[index_P]])
            df_KINEpart.loc[w,'Num_nonreco_protons'] = sum([True if Y <= threshold_P else False for Y in y[index_P]])
            df_KINEpart.loc[w,'Num_total_protons'] = sum([True for Y in y[index_P]])
            kine_reco_protons.append(y[index_P])
        else: kine_reco_protons.append([])
        #if (len(x) > 0) & (211 in x): 
        #    index_pi = [i for i, z in enumerate(x) if z == 211]
        #    num_pi = [True if Y > threshold_pi else False for Y in y[index_pi]]
        #    df_KINEpart.loc[w,'Num_pions'] = sum(num_pi)
        
    df_KINEpart.loc[:,'kine_reco_protons'] = kine_reco_protons
    ############################################################ Import T_BDTvars
    T_BDTvars = uproot.open(input_file)['wcpselection/T_BDTvars']
    df_BDTvars = T_BDTvars.pandas.df(BDT_variables, flatten=False) # BDT_variables contains only unique
    
    ############################################################ Import T_eval                                
    T_eval = uproot.open(input_file)['wcpselection/T_eval']
    if family=='MC':
        df_eval = T_eval.pandas.df(eval_variables_MC, flatten=False)
    elif family=='DATA':
        df_eval = T_eval.pandas.df(eval_variables_DATA, flatten=False)
    else: print(' -------- WARNING: Wrong family')
    
    ############################################################ Import T_PFeval + return
    if family=='MC':
        T_PFeval = uproot.open(input_file)['wcpselection/T_PFeval']
        df_PFeval = T_PFeval.pandas.df(pfeval_variables_PF, flatten=False)
        # Merge dataframes
        df = pd.concat([df_eval, df_KINEvars, df_KINEpart, df_BDTvars, df_PFeval], axis=1)
        # Limit weight values
        df['weight_cv'] = np.where((df.weight_cv <= 0), 1, df.weight_cv)
        df['weight_cv'] = np.where((df.weight_cv > 30), 1, df.weight_cv)
        df['weight_cv'] = np.where((df.weight_cv == np.nan), 1, df.weight_cv)
        df['weight_cv'] = np.where((df.weight_cv == np.inf), 1, df.weight_cv)
        df['weight_cv'] = np.where((df['weight_cv'].isna()), 1, df.weight_cv)
        df['weight_spline'] = np.where((df.weight_spline <= 0), 1, df.weight_spline)
        df['weight_spline'] = np.where((df.weight_spline > 30), 1, df.weight_spline)
        df['weight_spline'] = np.where((df.weight_spline == np.nan), 1, df.weight_spline)
        df['weight_spline'] = np.where((df.weight_spline == np.inf), 1, df.weight_spline)
        df['weight_spline'] = np.where((df['weight_spline'].isna()), 1, df.weight_spline)  
        
        df = add_reco_pi0_info(df)  # reco pi0 energy/momentum
        df = add_truth_pi0_info(df) # truth pi0 energy/momentum
        
        df.loc[:,'truth_Num_protons'] = [-1]*df.shape[0]
        for x,y,N in zip(df.truth_mother, df.truth_pdg, range(len(df.truth_mother))):
            df.loc[N,'truth_Num_protons'] = sum([1 for w,z in zip(list(x),list(y)) if (w == 0 and z == 2212)])
    
        #df = df.drop_duplicates(subset=['run','subrun','event']).reset_index(drop=True)
        print('Shape:', df.shape)
        return POT_file, df
    
    elif family=='DATA':
        # Merge dataframes
        df = pd.concat([df_eval, df_KINEvars, df_KINEpart, df_BDTvars], axis=1)
        
        df = add_reco_pi0_info(df)  # reco pi0 energy/momentum
        
        #df = df.drop_duplicates(subset=['run','subrun','event']).reset_index(drop=True)
        print('Shape:', df.shape)
        return POT_file, df
    
    else: print(' -------- WARNING: Wrong family')
        
def add_reco_pi0_info(df):
    pi0_mass = 135
    alpha = abs(df.kine_pio_energy_1 - df.kine_pio_energy_2)/(df.kine_pio_energy_1 + df.kine_pio_energy_2)
    df.loc[:,'reco_pi0_energy'] = pi0_mass * (np.sqrt(2./(1-alpha*alpha)/(1-np.cos(df.kine_pio_angle/180.*np.pi)))-1)
    df.loc[:,'reco_pi0_energy_tot'] = pi0_mass + df.reco_pi0_energy
    df.loc[:,'reco_pi0_momentum'] = np.sqrt(df.reco_pi0_energy_tot*df.reco_pi0_energy_tot - pi0_mass*pi0_mass)
    
    p_0 = df.kine_pio_energy_1 + df.kine_pio_energy_2
    # ----------------------------------------------------------
    p1_1 = df.kine_pio_energy_1*np.cos(df.kine_pio_phi_1/180.*np.pi)*np.sin(df.kine_pio_theta_1/180.*np.pi)
    p2_1 = df.kine_pio_energy_2*np.cos(df.kine_pio_phi_2/180.*np.pi)*np.sin(df.kine_pio_theta_2/180.*np.pi)
    p_1 = p1_1 + p2_1
    # ----------------------------------------------------------
    p1_2 = df.kine_pio_energy_1*np.sin(df.kine_pio_phi_1/180.*np.pi)*np.sin(df.kine_pio_theta_1/180.*np.pi)
    p2_2 = df.kine_pio_energy_2*np.sin(df.kine_pio_phi_2/180.*np.pi)*np.sin(df.kine_pio_theta_2/180.*np.pi)
    p_2 = p1_2 + p2_2
    # ----------------------------------------------------------
    p1_3 = df.kine_pio_energy_1*np.cos(df.kine_pio_theta_1/180.*np.pi)
    p2_3 = df.kine_pio_energy_2*np.cos(df.kine_pio_theta_2/180.*np.pi)
    p_3 = p1_3 + p2_3
    # ----------------------------------------------------------
    df.loc[:,'reco_pi0_costheta'] = p_3/np.sqrt(p_1**2 + p_2**2 + p_3**2)
    df.loc[:,'reco_E_nonpi0'] = df.kine_reco_Enu - df.reco_pi0_energy_tot
    df.loc[:,'reco_Epi0_Enu_ratio'] = df.reco_pi0_energy_tot/df.kine_reco_Enu
    df.loc[:,'reco_Epi0_cosine'] = df.reco_pi0_energy*(1. - df.reco_pi0_costheta)
    
    return df.reset_index(drop=True)
        
def add_truth_pi0_info(df):
    pi0_mass = 135
    alpha = abs(df.truth_pio_energy_1 - df.truth_pio_energy_2)/(df.truth_pio_energy_1 + df.truth_pio_energy_2)
    df.loc[:,'truth_pi0_energy'] = pi0_mass * (np.sqrt(2./(1-alpha*alpha)/(1-np.cos(df.truth_pio_angle/180.*np.pi)))-1)
    df.loc[:,'truth_pi0_energy_tot'] = pi0_mass + df.truth_pi0_energy
    df.loc[:,'truth_pi0_momentum'] = np.sqrt(df.truth_pi0_energy_tot*df.truth_pi0_energy_tot - pi0_mass*pi0_mass)
    #truth_pi0_Px = []
    #truth_pi0_Py = []
    #truth_pi0_Pz = []
    #truth_pi0_P = []
    truth_pi0_costheta = []
    for x,y,N,P in zip(df.truth_mother, df.truth_pdg, range(len(df.truth_mother)), df.truth_startMomentum):
        index = [1 if (w == 0 and z == 111) else 0 for w,z in zip(list(x),list(y))]
        ind = [i for i in range(len(index)) if index[i] == 1]
        if len(ind) > 0:
            P_pi = [P[j][3] for j in ind]
            ind = ind[P_pi.index(max(P_pi))]
            #truth_pi0_Px.append(P[ind][0])
            #truth_pi0_Py.append(P[ind][1])
            #truth_pi0_Pz.append(P[ind][2])
            #truth_pi0_P.append(P[ind][3])
            P_perp = np.sqrt(P[ind][0]*P[ind][0] + P[ind][1]*P[ind][1])
            truth_pi0_costheta.append(np.cos(np.arctan2(P_perp,P[ind][2])))#*180./np.pi)
        else:
            #truth_pi0_Px.append(-999.)
            #truth_pi0_Py.append(-999.)
            #truth_pi0_Pz.append(-999.)
            #truth_pi0_P.append(-999.)
            truth_pi0_costheta.append(-999.)
    #df.loc[:,'truth_pi0_Px'] = truth_pi0_Px
    #df.loc[:,'truth_pi0_Py'] = truth_pi0_Py
    #df.loc[:,'truth_pi0_Pz'] = truth_pi0_Pz
    #df.loc[:,'truth_pi0_P'] = truth_pi0_P
    df.loc[:,'truth_pi0_costheta'] = truth_pi0_costheta
    df.loc[:,'truth_Epi0_Enu_ratio'] = df.truth_pi0_energy_tot/df.truth_nuEnergy
    df.loc[:,'truth_Epi0_cosine'] = df.truth_pi0_energy*(1. - df.truth_pi0_costheta)
    return df.reset_index(drop=True)

In [4]:
def merge_files(list_df, list_POT, tag, family='MC'):
    # It works for one or multiple files
    # tag: it's just a printed info (Ext has a weight scaling)
    # family: MC / DATA
    
    POT_tot = sum(list_POT)
    
    #if tag == 'Ext': W_ = (POT_goal/(POT_tot*0.98)) # NuMI only
    if tag == 'Ext': W_ = POT_goal/POT_tot # NuMI only
    else: W_ = POT_goal/POT_tot
    
    print('%s: POT_tot: %s (Normalization factor: %1.4f)'%(tag, POT_tot, W_))
    
    df = pd.concat(list_df, axis=0).reset_index(drop=True)
    
    if family == 'MC':
        df.loc[:,'weight_genie'] = df['weight_cv']*df['weight_spline']
        df.loc[:,'weight'] = [W_]*df.shape[0] * df['weight_genie']
    elif family == 'DATA':
        df.loc[:,'weight'] = [W_]*df.shape[0]
    else: print(' -------- WARNING: Wrong family')
    
    # Check weights are defined
    print('Events (raw):',df.shape[0],' Events (weights):', sum(df.weight))
    
    return W_, df

def check_count(check, goal, weight=False):
    # Check sum of events when splitting or merging a DF, can use weights or raw numbers
    # check: list of DFs
    # goal: single DF (split or merged one)
    if weight==False:
        print('Check: %i = %i %s'%(goal.shape[0], sum([x.shape[0] for x in check]), [x.shape[0] for x in check]))
    elif weight==True: 
        print('Check: %1.1f = %1.1f %s'%(sum(goal.weight), sum([sum(x.weight) for x in check]), [round(sum(x.weight),1) for x in check]))


def gen_run_subrun_list(input_file, name_list):
    # Get useful variables
    T_eval = uproot.open(input_file)['wcpselection/T_eval']
    df_eval = T_eval.pandas.df(['run', 'subrun'], flatten=False)
    df_eval.drop_duplicates(inplace=True) 
    np.savetxt(name_list, df_eval.values, fmt='%d')

def split_train_val_test(df, tag):
    # Test is all events not labelled for training
    # Train is 3/4 of the remaining events, Validation is 1/4 of the remaining events
    df_test = df[df.Training_bool == False].reset_index(drop=True)
    df_ = shuffle(df[df.Training_bool == True]).reset_index(drop=True)
    df_val = df_.iloc[(df_.index % 4 == 0).astype(bool)].reset_index(drop=True)
    df_train = df_.iloc[(df_.index % 4 != 0).astype(bool)].reset_index(drop=True)
    print('Check (%s):'%(tag),df.shape[0],'=','(%s)'%(df_train.shape[0]+df_val.shape[0]+df_test.shape[0]),df_train.shape[0],'+',df_val.shape[0],'+',df_test.shape[0])
    return df_train, df_val, df_test
    
def split_train_val_test_fractions(df, tag):
    df_test = df.iloc[(df.index % 3 == 0).astype(bool)].reset_index(drop=True)
    df_train = df.iloc[(df.index % 3 != 0).astype(bool)].reset_index(drop=True)
    df_val = df_train.iloc[(df_train.index % 4 == 0).astype(bool)].reset_index(drop=True)
    df_train = df_train.iloc[(df_train.index % 4 != 0).astype(bool)].reset_index(drop=True)
    print('Check (%s):'%(tag),df.shape[0],'=','(%s)'%(df_train.shape[0]+df_val.shape[0]+df_test.shape[0]),df_train.shape[0],'+',df_val.shape[0],'+',df_test.shape[0])
    return df_train, df_val, df_test

def plot_important_features(features, feature_importances_, number, name):
    zipped = zip(features, feature_importances_)
    zipped_sort = sorted(zipped, key = lambda x:x[1], reverse=True)
    zipped_sort_reduced = zipped_sort[:number]
    
    res = [[ i for i, j in zipped_sort_reduced], 
           [ j for i, j in zipped_sort_reduced]]
    red_features = res[0]
    red_importances = res[1]
    
    plt.barh(range(len(red_importances)), red_importances, align='center')
    plt.yticks(np.arange(len(red_features)), red_features)
    plt.xlabel("Feature importance")
    plt.ylabel("Top %i features"%(number))
    #plt.xscale('log')
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.1)
    plt.tight_layout()
    #plt.savefig('%s.pdf'%(plot_name))
    
    return red_features

def return_bin_err(hist, weight, nbins, xrange):
    # hist: list containing the entries of the histogram
    # weight: list containing the weights of the entries
    # nbins: number of bins for the histogram
    # xrange: range of the histogram
    
    # Histogram w/out normalization (not shown)
    hist_, bins_ = np.histogram(hist, bins=nbins, range=xrange) 
    # Histogram w/ normalization (not shown)
    histw_, bins_ = np.histogram(hist, weights=weight, bins=nbins, range=xrange) 
    # Sum of the weights squared in each bin of the histogram
    sumw2 = []
    w = np.asarray(weight)
    for left, right in zip(bins_, bins_[1:]):
        ix = np.where((hist >= left) & (hist <= right))[0] 
        sumw2.append(np.sum(w[ix] ** 2))
    # Bin error for weighted histogram (Poisson)
    bin_err = [np.sqrt(x) if x > 0 else 0 for x in sumw2]
    # Return list of bin error bars for a weighted histogram
    return bin_err

def bdtscore_cut_eff(hist, weight, nbins, xrange, efficiency=True):
    # hist: list containing the entries of the histogram
    # weight: list containing the weights of the entries
    # nbins: number of bins for the histogram
    # xrange: range of the histogram
    # efficiency: bool parameter, if True efficiency list, if False cumulative list
    
    # Histogram w/ normalization (not shown)
    hist_, bins_ = np.histogram(hist, weights=weight, bins=nbins, range=xrange)
    # Bin error for weighted histogram (Poisson)
    bin_err = return_bin_err(hist, weight, nbins, xrange)
    # List with sum of weights squared in each bin
    sumw2 = [x**2 for x in bin_err]
    # Weighted total integral
    h_integral = sum(hist_)
    # List with cut values
    step = (xrange[1]-xrange[0])/nbins
    cut_values = [xrange[0]+(x*step) for x in range(nbins)]
    # List with cumulative histogram up to cut value
    cumul_cut = []
    # Append cumulative to the right of the value
    for ith in range(len(cut_values)):
        if ith == 0: cumul_cut.append(sum(hist_))
        elif ith > 0: cumul_cut.append(sum(hist_[ith:]))
    # Return efficiency list or cumulative list
    if efficiency==True: return [x/h_integral for x in cumul_cut]
    elif efficiency==False: return cumul_cut
    
def return_cumulative_hist(hists, weights):
    cum = []
    for sublist in hists:
        for item in sublist: cum.append(item)        
    weight_cum = []
    for sublist in weights:
        for item in sublist: weight_cum.append(item)
    return cum, weight_cum

def return_used_list(df, df_runsub):
    df_found = pd.merge(df, df_runsub, on=['run','subrun'], how='left', indicator='Used')
    df_found['Used'] = np.where(df_found.Used == 'both', True, False)
    return df_found['Used']


def xin_transf_scalar(x):
    if x <= 0: return math.log10((1.+np.tanh(logit(1e-5)))/(1.-np.tanh(logit(1e-5))))
    elif x >= 1: return math.log10((1.+np.tanh(logit(0.9999)))/(1.-np.tanh(logit(0.9999))))
    else: return math.log10((1.+np.tanh(logit(x)))/(1.-np.tanh(logit(x))))

def xin_transf(x):
    if type(x) == float or type(x) == int: return xin_transf_scalar(x)
    elif type(x) == list: return [xin_transf_scalar(y) for y in x]
    

def two_panel_plot(hists, weights, nbins, xrange, yrange, colors, labels, xlabels, ylabels, save_name=None):
    # hists: list with lists of histograms [list(lists)]
    # weights: list with lists of weights [list(lists)]
    # nbins: list with number/list of bins in each panel [list(int/list)]
    # xrange: list with x-axis range in each panel [list(tuple)]
    # colors: list with lists of colors [list(lists)]
    # labels: list with lists of labels [list(lists)]
    # xlabels: list with x-labels [list(str)]
    # ylabels: list with y-labels [list(str)]
    
    histsA, histsB = hists[0], hists[1]
    weightsA, weightsB = weights[0], weights[1]
    nbinsA, nbinsB = nbins[0], nbins[1]
    xrangeA, xrangeB = xrange[0], xrange[1]
    yrangeA, yrangeB = yrange[0], yrange[1]
    colorsA, colorsB = colors[0], colors[1]
    labelsA, labelsB = labels[0], labels[1]
    xlabelA, xlabelB = xlabels[0], xlabels[1]
    ylabelA, ylabelB = ylabels[0], ylabels[1]
    
    hist_dataA, hist_dataB = histsA[0], histsB[0]
    w_dataA, w_dataB = weightsA[0], weightsB[0]
    c_dataA, c_dataB = colorsA[0], colorsB[0]
    label_dataA, label_dataB = labelsA[0], labelsB[0]
    
    hist_listA, hist_listB = histsA[1:], histsB[1:]
    w_listA, w_listB = weightsA[1:], weightsB[1:]
    c_listA, c_listB = colorsA[1:], colorsB[1:]
    label_listA, label_listB = labelsA[1:], labelsB[1:]
    
    fig = plt.figure(figsize=(15,6))
    gs = gridspec.GridSpec(1, 2, figure=fig, wspace=0.08)
    # ---------------------------------------------------------------------------------------------------
    # Panel A (left)
    gsA = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=gs[0], hspace=0.07, height_ratios=(3.5,1))
    # ---------------------------------------------------------------------------------------------------
    # Panel A top (left, top)
    ax1 = fig.add_subplot(gsA[0,0])
    ax1.hist(hist_listA, weights=w_listA, bins=nbinsA, range=xrangeA, color=c_listA, label=label_listA, stacked=True)
    data_histA, binsA_ = np.histogram(hist_dataA, weights=w_dataA, bins=nbinsA, range=xrangeA)
    data_errbarA = return_bin_err(hist_dataA, w_dataA, nbinsA, xrangeA)
    midA = 0.5*(binsA_[1:] + binsA_[:-1])
    if type(nbinsA) == int: xerrA = 0.5*xrangeA[1]/nbinsA
    else: xerrA = [(x-y)/2 for x,y in zip(nbinsA[1:],nbinsA)]
    ax1.errorbar(midA, data_histA, xerr=xerrA, yerr=data_errbarA, color=c_dataA, label=label_dataA, fmt='o')
    ax1.axes.xaxis.set_ticklabels([])
    ax1.set_ylabel(ylabelA)
    ax1.set_xlim(xrangeA)
    ax1.set_ylim(yrangeA)
    ax1.legend(loc='best', ncol=1)#, prop={'size': legend_size})
    # ---------------------------------------------------------------------------------------------------
    # Panel A bottom (left, bottom)
    ax2 = fig.add_subplot(gsA[1,0])
    cumA, weight_cumA = return_cumulative_hist(hist_listA, w_listA)
    pred_histA, binsA_ = np.histogram(cumA, weights=weight_cumA, bins=nbinsA, range=xrangeA)
    pred_errbarA = return_bin_err(cumA, weight_cumA, nbinsA, xrangeA)
    ratioA = [x/y if y > 0 else np.nan for x,y in zip(data_histA, pred_histA)]
    ratio_effA = [np.sqrt((x**2)*((a/A)**2+(b/B)**2)) if A*B > 0 else np.nan for x,a,A,b,B in zip(ratioA, data_errbarA, data_histA, pred_errbarA, pred_histA)]
    ax2.errorbar(midA, ratioA, xerr=xerrA, yerr=ratio_effA, color=c_dataA, fmt='o')
    ax2.hlines(1, xrangeA[0], xrangeA[1], ls='--', color='black', lw=1, alpha=0.5)
    ax2.set_xlabel(xlabelA)
    ax2.set_ylabel('Data/Pred')
    ax2.set_xlim(xrangeA)
    ax2.set_ylim((0,2))
    chi2A, p_valA = chisquare(data_histA, pred_histA)[0], chisquare(data_histA, pred_histA)[1] 
    print('Panel Left: Chi-square = %1.4f, p-value = %1.4f'%(chi2A, p_valA))
    # ---------------------------------------------------------------------------------------------------
    # Panel B (right)
    gsB = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=gs[1], hspace=0.07, height_ratios=(3.5,1))
    # ---------------------------------------------------------------------------------------------------
    # Panel B top (right, top)
    ax3 = fig.add_subplot(gsB[0,0])
    ax3.hist(hist_listB, weights=w_listB, bins=nbinsB, range=xrangeB, color=c_listB, label=label_listB, stacked=True)
    data_histB, binsB_ = np.histogram(hist_dataB, weights=w_dataB, bins=nbinsB, range=xrangeB)
    data_errbarB = return_bin_err(hist_dataB, w_dataB, nbinsB, xrangeB)
    midB = 0.5*(binsB_[1:] + binsB_[:-1])
    if type(nbinsB) == int: xerrB = 0.5*xrangeB[1]/nbinsB
    else: xerrB = [(x-y)/2 for x,y in zip(nbinsB[1:],nbinsB)]
    ax3.errorbar(midB, data_histB, xerr=xerrB, yerr=data_errbarB, color=c_dataB, label=label_dataB, fmt='o')
    ax3.axes.xaxis.set_ticklabels([])
    #ax3.set_ylabel(ylabelB)
    ax3.set_xlim(xrangeB)
    ax3.set_ylim(yrangeB)
    ax3.legend(loc='best', ncol=1)#, prop={'size': legend_size})
    # ---------------------------------------------------------------------------------------------------
    # Panel B bottom (right, bottom)
    ax4 = fig.add_subplot(gsB[1,0])
    cumB, weight_cumB = return_cumulative_hist(hist_listB, w_listB)
    pred_histB, binsB_ = np.histogram(cumB, weights=weight_cumB, bins=nbinsB, range=xrangeB)
    pred_errbarB = return_bin_err(cumB, weight_cumB, nbinsB, xrangeB)
    ratioB = [x/y if y > 0 else np.nan for x,y in zip(data_histB, pred_histB)]
    ratio_effB = [np.sqrt((x**2)*((a/A)**2+(b/B)**2)) if A*B > 0 else np.nan for x,a,A,b,B in zip(ratioB, data_errbarB, data_histB, pred_errbarB, pred_histB)]
    ax4.errorbar(midB, ratioB, xerr=xerrB, yerr=ratio_effB, color=c_dataB, fmt='o')
    ax4.hlines(1, xrangeB[0], xrangeB[1], ls='--', color='black', lw=1, alpha=0.5)
    ax4.set_xlabel(xlabelB)
    #ax4.set_ylabel('Data/Pred')
    ax4.set_xlim(xrangeB)
    ax4.set_ylim((0,2))
    chi2B, p_valB = chisquare(data_histB, pred_histB)[0], chisquare(data_histB, pred_histB)[1] 
    print('Panel Right: Chi-square = %1.4f, p-value = %1.4f'%(chi2B, p_valB))
    # ---------------------------------------------------------------------------------------------------
    if save_name: plt.savefig(save_name)
    plt.show()

def one_panel_plot(hists, weights, nbins, xrange, yrange, colors, labels, xlabel, ylabel, save_name=None, yscale=None):
    # hists: list with lists of histograms [list(lists)]
    # weights: list with lists of weights [list(lists)]
    # nbins: list with number/list of bins in each panel [list(int/list)]
    # xrange: list with x-axis range in each panel [list(tuple)]
    # colors: list with lists of colors [list(lists)]
    # labels: list with lists of labels [list(lists)]
    # xlabels: list with x-labels [list(str)]
    # ylabels: list with y-labels [list(str)]
    
    hist_data = hists[0]
    w_data = weights[0]
    c_data = colors[0]
    label_data = labels[0]
    
    hist_list = hists[1:]
    w_list = weights[1:]
    c_list = colors[1:]
    label_list = labels[1:]
    
    fig = plt.figure(figsize=(11,8.6))
    gs = gridspec.GridSpec(2, 1, hspace=0.07, height_ratios=(3.5,1))
    # ---------------------------------------------------------------------------------------------------
    # Panel top 
    ax1 = fig.add_subplot(gs[0])
    ax1.hist(hist_list, weights=w_list, bins=nbins, range=xrange, color=c_list, label=label_list, stacked=True)
    data_hist, bins_ = np.histogram(hist_data, weights=w_data, bins=nbins, range=xrange)
    data_errbar = return_bin_err(hist_data, w_data, nbins, xrange)
    mid = 0.5*(bins_[1:] + bins_[:-1])
    if type(nbins) == int: xerr = 0.5*xrange[1]/nbins
    else: xerr = [(x-y)/2 for x,y in zip(nbins[1:],nbins)]
    ax1.errorbar(mid, data_hist, xerr=xerr, yerr=data_errbar, color=c_data, label=label_data, fmt='o')
    ax1.axes.xaxis.set_ticklabels([])
    if yscale: ax1.set_yscale(yscale)
    ax1.set_ylabel(ylabel)
    ax1.set_xlim(xrange)
    ax1.set_ylim(yrange)
    ax1.legend(loc='best', ncol=2)#, prop={'size': legend_size})
    # ---------------------------------------------------------------------------------------------------
    # Panel bottom
    ax2 = fig.add_subplot(gs[1])
    cum, weight_cum = return_cumulative_hist(hist_list, w_list)
    pred_hist, bins_ = np.histogram(cum, weights=weight_cum, bins=nbins, range=xrange)
    pred_errbar = return_bin_err(cum, weight_cum, nbins, xrange)
    ratio = [x/y if y > 0 else np.nan for x,y in zip(data_hist, pred_hist)]
    ratio_eff = [np.sqrt((x**2)*((a/A)**2+(b/B)**2)) if A*B > 0 else np.nan for x,a,A,b,B in zip(ratio, data_errbar, data_hist, pred_errbar, pred_hist)]
    ax2.errorbar(mid, ratio, xerr=xerr, yerr=ratio_eff, color=c_data, fmt='o')
    ax2.hlines(1, xrange[0], xrange[1], ls='--', color='black', lw=1, alpha=0.5)
    ax2.set_xlabel(xlabel)
    ax2.set_ylabel('Data/Pred')
    ax2.set_xlim(xrange)
    ax2.set_ylim((0,2))
    chi2, p_val = chisquare(data_hist, pred_hist)[0], chisquare(data_hist, pred_hist)[1] 
    print('Chi-square = %1.4f, p-value = %1.4f'%(chi2, p_val))
    # ---------------------------------------------------------------------------------------------------
    gs.tight_layout(fig)
    if save_name: plt.savefig(save_name)
    plt.show()

def hist_compare_pdf(hists, weights, nbins, xrange, yrange, colors, labels, xlabel, ylabel, save_name=None):
    hist1,hist2 = hists[0],hists[1]
    w1,w2 = weights[0],weights[1]
    c1,c2 = colors[0],colors[1]
    l1,l2 = labels[0],labels[1]
    
    fig = plt.figure(figsize=(9,7))
    gs = gridspec.GridSpec(2, 1, hspace=0.07, height_ratios=(3.5,1))
    # ---------------------------------------------------------------------------------------------------
    # Panel top 
    ax1 = fig.add_subplot(gs[0])
    ax1.hist(hist1, weights=w1, bins=nbins, range=xrange, color=c1, label=l1, alpha=0.5, density=True)
    ax1.hist(hist2, weights=w2, bins=nbins, range=xrange, color=c2, label=l2, alpha=0.5, density=True)
    data1, bins_ = np.histogram(hist1, weights=w1, bins=nbins, range=xrange, density=True)
    data2, bins_ = np.histogram(hist2, weights=w2, bins=nbins, range=xrange, density=True)
    mid = 0.5*(bins_[1:] + bins_[:-1])
    ax1.axes.xaxis.set_ticklabels([])
    ax1.set_ylabel(ylabel)
    ax1.set_xlim(xrange)
    ax1.set_ylim(yrange)
    ax1.legend(loc='best', ncol=1)#, prop={'size': legend_size})
    # ---------------------------------------------------------------------------------------------------
    # Panel bottom
    ax2 = fig.add_subplot(gs[1])
    ratio = [x/y if y > 0 else np.nan for x,y in zip(data1, data2)]
    ax2.errorbar(mid, ratio, xerr=0, yerr=0, color='black', fmt='o')
    ax2.hlines(1, xrange[0], xrange[1], ls='--', color='black', lw=1, alpha=0.5)
    ax2.set_xlabel(xlabel)
    ax2.set_ylabel('BDT/Cut-based')
    ax2.set_xlim(xrange)
    ax2.set_ylim((0,2))
    # ---------------------------------------------------------------------------------------------------
    if save_name: plt.savefig(save_name)
    plt.show()

# EFFICIENCY AND PURITY WITH ERROR PROPAGATION
def return_sumw2(x, w, bins):
    sumw2 = []
    for left, right in zip(bins, bins[1:]): 
        ix = np.where((x >= left) & (x <= right))[0] 
        sumw2.append(np.sum(w[ix] ** 2))
    return sumw2

def return_hist_delta(x, w, bins, xrange):
    hist_, bins_, _ = plt.hist(x, bins=bins, range=xrange, alpha=0)
    hist_hat_, bins_, _ = plt.hist(x, weights=w, bins=bins, range=xrange, alpha=0)  
    sumw2 = return_sumw2(x, np.asarray(w), bins_)
    delta = [np.sqrt(x) if x > 0 else 0 for x in sumw2]
    return hist_hat_, delta, bins_

def propagate_purity(S, delta_S, B, delta_B, bins):
    # purity for each bin
    ratio_list = [x/(x+y) if x+y > 0 else np.nan for x,y in zip(S, B)]
    yerr_list = []
    
    n_bins = 0
    if type(bins) == int: n_bins = bins
    elif type(bins) != int: n_bins = len(bins)-1

    # uncertainty propagation for S/(S+B)
    for ith in range(n_bins):
        if S[ith]+B[ith] > 0: yerr_list.append(np.sqrt(((B[ith]*delta_S[ith])**2 + (S[ith]*delta_B[ith])**2)/(S[ith] + B[ith])**4))
        else: yerr_list.append(0)
            
    return ratio_list, yerr_list

def propagate_purity_lee(S, delta_S, B, delta_B, w_lee, bins):
    # purity for each bin
    ratio_list = [w*x/((1.+w)*x+y) if (1.+w)*x+y > 0 else np.nan for x,y,w in zip(S, B, w_lee)]
    yerr_list = []
    
    n_bins = 0
    if type(bins) == int: n_bins = bins
    elif type(bins) != int: n_bins = len(bins)-1

    # uncertainty propagation for w*S/((1.+w)*S+B)
    for ith in range(n_bins):
        if (1.+w_lee[ith])*S[ith]+B[ith] > 0: yerr_list.append(np.sqrt(((w_lee[ith]*B[ith]*delta_S[ith])**2 + (w_lee[ith]*S[ith]*delta_B[ith])**2)/((1.+w_lee[ith])*S[ith]+B[ith])**4))
        else: yerr_list.append(0)
            
    return ratio_list, yerr_list

# --------------------------------------------------------------------- PURITY PLOT

def return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color, fmt='none', label=None):
    n_bins = 0
    if type(bins) == int: n_bins = bins
    elif type(bins) != int: n_bins = len(bins)-1
    
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  Signal hist
    S_hat_hist, delta_S_hat, bins_ = return_hist_delta(hist, weight, bins, xrange)
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  Background hists
    # EXT
    if len(back_hists[0]) > 0:
        B_ext_hat_hist, delta_B_ext_hat, bins_ = return_hist_delta(back_hists[0], back_weights[0], bins, xrange)
    else:
        B_ext_hat_hist = [0.]*n_bins
        delta_B_ext_hat = [0.]*n_bins
    # OVERLAY
    if len(back_hists[1]) > 0:
        B_over_hat_hist, delta_B_over_hat, bins_ = return_hist_delta(back_hists[1], back_weights[1], bins, xrange)
    else:
        B_over_hat_hist = [0.]*n_bins
        delta_B_over_hat = [0.]*n_bins
    # INTRINSIC
    if len(back_hists[2]) > 0:
        B_int_hat_hist, delta_B_int_hat, bins_ = return_hist_delta(back_hists[2], back_weights[2], bins, xrange)
    else:
        B_int_hat_hist = [0.]*n_bins
        delta_B_int_hat = [0.]*n_bins
   # DIRT
    if len(back_hists[3]) > 0:
        B_dirt_hat_hist, delta_B_dirt_hat, bins_ = return_hist_delta(back_hists[3], back_weights[3], bins, xrange)
    else:
        B_dirt_hat_hist = [0.]*n_bins
        delta_B_dirt_hat = [0.]*n_bins
    
    B_hat_hist = []
    delta_B_hat = []    
    for ith in range(n_bins):
        B_hat_hist.append(B_ext_hat_hist[ith] + B_over_hat_hist[ith] + B_int_hat_hist[ith] + B_dirt_hat_hist[ith])
        delta_B_hat.append(np.sqrt((delta_B_ext_hat[ith])**2 + 
                                   (delta_B_over_hat[ith])**2 + 
                                   (delta_B_int_hat[ith])**2 + 
                                   (delta_B_dirt_hat[ith])**2))
    
    ratio_list, yerr_list = propagate_purity(S_hat_hist, delta_S_hat, B_hat_hist, delta_B_hat, bins)

    mid=0.5*(bins_[1:] + bins_[:-1])
    xerr_list = [(x-y)/2. for x,y in zip(bins_[1:], bins_)]
    
    if label != None: return plt.errorbar(x=mid, y=ratio_list, xerr=xerr_list, yerr=yerr_list, c=color, fmt=fmt, label=label)
    else: return plt.errorbar(x=mid, y=ratio_list, xerr=xerr_list, yerr=yerr_list, c=color, fmt=fmt)


def return_purity_lee_plot(hist, weight, back_hists, back_weights, w_lee, bins, xrange, color, fmt='none', label=None):
    n_bins = 0
    if type(bins) == int: n_bins = bins
    elif type(bins) != int: n_bins = len(bins)-1
    
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  Signal hist
    S_hat_hist, delta_S_hat, bins_ = return_hist_delta(hist, weight, bins, xrange)
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  Background hists
    # EXT
    if len(back_hists[0]) > 0:
        B_ext_hat_hist, delta_B_ext_hat, bins_ = return_hist_delta(back_hists[0], back_weights[0], bins, xrange)
    else:
        B_ext_hat_hist = [0.]*n_bins
        delta_B_ext_hat = [0.]*n_bins
    # OVERLAY
    if len(back_hists[1]) > 0:
        B_over_hat_hist, delta_B_over_hat, bins_ = return_hist_delta(back_hists[1], back_weights[1], bins, xrange)
    else:
        B_over_hat_hist = [0.]*n_bins
        delta_B_over_hat = [0.]*n_bins
    # INTRINSIC
    if len(back_hists[2]) > 0:
        B_int_hat_hist, delta_B_int_hat, bins_ = return_hist_delta(back_hists[2], back_weights[2], bins, xrange)
    else:
        B_int_hat_hist = [0.]*n_bins
        delta_B_int_hat = [0.]*n_bins
   # DIRT
    if len(back_hists[3]) > 0:
        B_dirt_hat_hist, delta_B_dirt_hat, bins_ = return_hist_delta(back_hists[3], back_weights[3], bins, xrange)
    else:
        B_dirt_hat_hist = [0.]*n_bins
        delta_B_dirt_hat = [0.]*n_bins
    
    B_hat_hist = []
    delta_B_hat = []    
    for ith in range(n_bins):
        B_hat_hist.append(B_ext_hat_hist[ith] + B_over_hat_hist[ith] + B_int_hat_hist[ith] + B_dirt_hat_hist[ith])
        delta_B_hat.append(np.sqrt((delta_B_ext_hat[ith])**2 + 
                                   (delta_B_over_hat[ith])**2 + 
                                   (delta_B_int_hat[ith])**2 + 
                                   (delta_B_dirt_hat[ith])**2))
    
    ratio_list, yerr_list = propagate_purity_lee(S_hat_hist, delta_S_hat, B_hat_hist, delta_B_hat, w_lee, bins)

    mid=0.5*(bins_[1:] + bins_[:-1])
    xerr_list = [(x-y)/2. for x,y in zip(bins_[1:], bins_)]
    
    if label != None: return plt.errorbar(x=mid, y=ratio_list, xerr=xerr_list, yerr=yerr_list, c=color, fmt=fmt, label=label)
    else: return plt.errorbar(x=mid, y=ratio_list, xerr=xerr_list, yerr=yerr_list, c=color, fmt=fmt)

# --------------------------------------------------------------------- EFFICIENCY PLOT

def return_eff_plot(hist, weight, back_hist, back_weight, bins, xrange, color, fmt='none', label=None, verbose=None):
    S_hist, delta_S, bins_ = return_hist_delta(hist, weight, bins, xrange)
    B_hist, delta_B, bins_ = return_hist_delta(back_hist, back_weight, bins, xrange)
    
    ratio_list, yerr_list = propagate_purity(S_hist, delta_S, B_hist, delta_B, bins)

    mid=0.5*(bins_[1:] + bins_[:-1])
    xerr_list = [(x-y)/2. for x,y in zip(bins_[1:], bins_)]
    
    if verbose:
        for x,cv,err in zip(range(len(mid)), ratio_list, yerr_list):
            if cv >= 0.: print('Bin %i = %1.3f +- %1.3f'%(x, cv, err))
        print('---------------------\n')
    return plt.errorbar(x=mid, y=ratio_list, xerr=xerr_list, yerr=yerr_list, c=color, fmt=fmt, label=label)

### Import files 

In [5]:
file_over1 = '/home/gs627/wc_frame/LEEana/processed_checkout_rootfiles/CV/checkout_prodgenie_bnb_nu_overlay_run1.root'
#file_over2 = '/home/gs627/wc_frame/LEEana/processed_checkout_rootfiles/CV/checkout_prodgenie_bnb_nu_overlay_run2_PF.root'
file_over3 = '/home/gs627/wc_frame/LEEana/processed_checkout_rootfiles/CV/checkout_prodgenie_bnb_nu_overlay_run3.root'

file_fakeV = '/home/gs627/wc_frame/LEEana/processed_checkout_rootfiles/CV/checkout_nu2020_fakedata_set5.root'

file_fakeVII1 = '/home/gs627/wc_frame/LEEana/processed_checkout_rootfiles/CV/checkout_fakedata_set7_run1.root'
file_fakeVII3 = '/home/gs627/wc_frame/LEEana/processed_checkout_rootfiles/CV/checkout_fakedata_set7_run3.root'

### Merge and pre-process samples

In [6]:
POT_goal = 7.243244e+20 

In [7]:
# Fake V (data but MC)
POT1_fake, df1_fake = gen_dataframe(file_fakeV, 'Fake V', family='MC', POT_file=POT_goal)

W_fake, df_fake = merge_files([df1_fake], 
                                [POT1_fake], 'Fake V', family='MC')

check_count(check=[df1_fake], goal=df_fake, weight=False)

Fake V [MC]: POT = 7.243244e+20


KeyError: not found: b'truth_isCC'
 in file: /home/gs627/wc_frame/LEEana/processed_checkout_rootfiles/CV/checkout_nu2020_fakedata_set5.root

In [ ]:
# MC
POT1_over, df1_over = gen_dataframe(file_over1, 'BNB overlay Run 1', family='MC')
POT3_over, df3_over = gen_dataframe(file_over3, 'BNB overlay Run 3', family='MC')

W_over, df_over = merge_files([df1_over, df3_over], 
                              [POT1_over, POT3_over], 'BNB overlay', family='MC') 

check_count(check=[df1_over, df3_over], goal=df_over, weight=False)

In [ ]:
# Indicative map of event subdivision
'''
data ----> data
over ----> (outFV_over, badmatch_over, NCinFV, NCpi0inFV, numuCCinFV, CCpi0inFV)

outFV if merged ------> outFV
badmatch if merged ---> cosmics 
'''

def is_outFV(df):
    df_ = df[((df.match_completeness_energy > 0.1*df.truth_energyInside) & 
              (df.truth_vtxInside == 0))]
    return df_

def is_badmatch(df):
    df_ = df[(df.match_completeness_energy <= 0.1*df.truth_energyInside)]
    return df_
    
def is_nueCCinFV(df):
    df_ = df[((df.match_completeness_energy > 0.1*df.truth_energyInside) & 
              (abs(df.truth_nuPdg) == 12) & 
              (df.truth_isCC == 1) &
              (df.truth_vtxInside == 1))]
    return df_

def is_NCinFV(df):
    df_ = df[((df.match_completeness_energy > 0.1*df.truth_energyInside) & 
              (df.truth_isCC == 0) &
              (df.truth_vtxInside == 1) &
              (df.truth_NprimPio == 0))]
    return df_

def is_NCpi0inFV(df):
    df_ = df[((df.match_completeness_energy > 0.1*df.truth_energyInside) & 
              (df.truth_isCC == 0) &
              (df.truth_vtxInside == 1) &
              (df.truth_NprimPio > 0))]
    return df_

def is_NCpi0inFV_efficiency(df):
    df_ = df[((df.truth_isCC == 0) &
              (df.truth_vtxInside == 1) &
              (df.truth_NprimPio > 0))]
    return df_


def is_numuCCinFV(df):
    df_ = df[((df.match_completeness_energy > 0.1*df.truth_energyInside) & 
              (abs(df.truth_nuPdg) == 14) &
              (df.truth_isCC == 1) &
              (df.truth_vtxInside == 1) &
              (df.truth_NprimPio == 0))]
    return df_
    
def is_CCpi0inFV(df):
    df_ = df[((df.match_completeness_energy > 0.1*df.truth_energyInside) & 
              (abs(df.truth_nuPdg) == 14) &
              (df.truth_isCC == 1) &
              (df.truth_vtxInside == 1) &
              (df.truth_NprimPio > 0))]
    return df_

def is_0p(df):
    df_ = df[df.Num_protons == 0]
    return df_

def is_Np(df):
    df_ = df[df.Num_protons > 0]
    return df_

def is_fake_0p(df):
    df_ = df[df.Num_total_protons == 0]
    return df_

def is_fake_Np(df):
    df_ = df[df.Num_total_protons > 0]
    return df_

def is_FC(df):
    df_ = df[df.match_isFC == 1]
    return df_

def is_PC(df):
    df_ = df[df.match_isFC == 0]
    return df_

def is_NCpi0BDT(df):
    df_ = df[((df.nc_pio_score > 1.816) &   #(df.ncpi0_score > 0.89) & # 0.89 (if imported with xgboost), 1.816 with Xin's scores
              (df.numu_cc_flag >= 0) &
              (df.kine_pio_energy_1 > 0) &
              (df.kine_pio_energy_2 > 0))]
    return df_

def isnot_NCpi0BDT(df):
    df_ = df[~((df.nc_pio_score > 1.816) &   #(df.ncpi0_score > 0.89) & # 0.89 (if imported with xgboost), 1.816 with Xin's scores
               (df.numu_cc_flag >= 0) &
               (df.kine_pio_energy_1 > 0) &
               (df.kine_pio_energy_2 > 0))]
    return df_

def isnot_generic(df):
    df_ = df[~((df.match_found == 1) & 
               (df.stm_eventtype != 0) &
               (df.stm_lowenergy == 0) &
               (df.stm_LM == 0) &
               (df.stm_TGM == 0) &
               (df.stm_STM == 0) &
               (df.stm_FullDead == 0) &
               (df.stm_clusterlength > 15))]
    return df_

In [ ]:
df_fake_NCpi0inFV = is_NCpi0inFV(df_fake)
df_NCpi0inFV = is_NCpi0inFV(df_over)

In [ ]:
colors =   ['black', 'royalblue']
labels =   ['Fake V', r'NC $\pi^{0}$ in FV']
families = ['MC', 'MC']
samples =  [df_fake, df_NCpi0inFV]
#samples =  [is_NCpi0BDT(is_generic(x)) for x in samples]

In [ ]:
MIN = 0         
MAX = 1500      
BINS = [0,MAX] 

hist_energy = [y.kine_reco_Enu for y in [is_NCpi0BDT(is_generic(x)) for x in samples]]
w_ = [y[y.kine_reco_Enu < MAX].weight for y in [is_NCpi0BDT(is_generic(x)) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_energy, w_, BINS, (MIN,MAX), (0,1000), colors, l_, r'Reco $\nu$ energy [MeV]', 'Event counts', 
               save_name='plot_open_dataset/ALL_kine_reco_Enu.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
POT_goal = 1.82202e+20 + 2.01956e+20

In [ ]:
hist_1 = [y[y.kine_pio_energy_1 < 400].kine_pio_energy_1 for y in [is_generic(x) for x in samples]]
w_ = [y[y.kine_pio_energy_1 < 400].weight for y in [is_generic(x) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_1, w_, 20, (0,400), (0,18000), colors, l_, r'kine_pio_energy_1 [MeV]', 'Event counts', yscale='log',
               save_name='plot_open_dataset/BDTvar_kine_pio_energy_1.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_1 = [y[y.kine_pio_energy_2 < 400].kine_pio_energy_2 for y in [is_generic(x) for x in samples]]
w_ = [y[y.kine_pio_energy_2 < 400].weight for y in [is_generic(x) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_1, w_, 20, (0,400), (0,18000), colors, l_, r'kine_pio_energy_2 [MeV]', 'Event counts', yscale='log',
               save_name='plot_open_dataset/BDTvar_kine_pio_energy_2.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_1 = [y[y.lem_shower_num_segs < 150].lem_shower_num_segs for y in [is_generic(x) for x in samples]]
w_ = [y[y.lem_shower_num_segs < 150].weight for y in [is_generic(x) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_1, w_, 15, (0,150), (0,20000), colors, l_, r'lem_shower_num_segs [MeV]', 'Event counts', yscale='log',
               save_name='plot_open_dataset/BDTvar_lem_shower_num_segs.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_1 = [y[y.numu_score < 9].numu_score for y in [is_generic(x) for x in samples]]
w_ = [y[y.numu_score < 9].weight for y in [is_generic(x) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_1, w_, 20, (-7,9), (0,10000), colors, l_, r'numu_score', 'Event counts', yscale='log',
               save_name='plot_open_dataset/BDTvar_numu_score.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_1 = [y[y.kine_pio_mass < 360].kine_pio_mass for y in [is_generic(x) for x in samples]]
w_ = [y[y.kine_pio_mass < 360].weight for y in [is_generic(x) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_1, w_, 20, (0,360), (0,18000), colors, l_, r'kine_pio_mass [MeV/c$^{2}$]', 'Event counts', yscale='log',
               save_name='plot_open_dataset/BDTvar_kine_pio_mass.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_1 = [y[y.br1_2_max_length < 400].br1_2_max_length for y in [is_generic(x) for x in samples]]
w_ = [y[y.br1_2_max_length < 400].weight for y in [is_generic(x) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_1, w_, 20, (0,400), (0,20000), colors, l_, r'br1_2_max_length [cm]', 'Event counts', yscale='log',
               save_name='plot_open_dataset/BDTvar_br1_2_max_length.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_1 = [y[y.brm_acc_direct_length < 400].brm_acc_direct_length for y in [is_generic(x) for x in samples]]
w_ = [y[y.brm_acc_direct_length < 400].weight for y in [is_generic(x) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_1, w_, 20, (0,400), (0,20000), colors, l_, r'brm_acc_direct_length [cm]', 'Event counts', yscale='log',
               save_name='plot_open_dataset/BDTvar_brm_acc_direct_length.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_1 = [y[y.kine_pio_vtx_dis < 400].kine_pio_vtx_dis for y in [is_generic(x) for x in samples]]
w_ = [y[y.kine_pio_vtx_dis < 400].weight for y in [is_generic(x) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_1, w_, 20, (0,400), (0,6000), colors, l_, r'kine_pio_vtx_dis [cm]', 'Event counts', yscale='log',
               save_name='plot_open_dataset/BDTvar_kine_pio_vtx_dis.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_1 = [y[y.anc_flag_main_outside < 2].anc_flag_main_outside for y in [is_generic(x) for x in samples]]
w_ = [y[y.anc_flag_main_outside < 2].weight for y in [is_generic(x) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_1, w_, 2, (0,1), (0,20000), colors, l_, r'anc_flag_main_outside', 'Event counts', yscale='log',
               save_name='plot_open_dataset/BDTvar_anc_flag_main_outside.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

## Open Data Set NCpi0BDT Distributions

In [ ]:
MIN = 0         #0
MAX = 8000      #1600
BINS = [0,8000] #24

hist_energy = [y[y.kine_reco_Enu < MAX].kine_reco_Enu for y in [is_NCpi0BDT(is_generic(x)) for x in samples]]
w_ = [y[y.kine_reco_Enu < MAX].weight for y in [is_NCpi0BDT(is_generic(x)) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_energy, w_, BINS, (MIN,MAX), (0,1000), colors, l_, r'Reco $\nu$ energy [MeV]', 'Event counts', 
               save_name='plot_open_dataset/ALL_kine_reco_Enu.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/(MC+EXT): %1.2f / %1.2f = %1.2f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
MIN = 0                                              #0
MAX = 4000                                           #1000 
BINS = [0,150,300,450,600,1000,2000,3000,4000]       #25 

hist_pi0momentum = [y[y.reco_pi0_momentum < MAX].reco_pi0_momentum for y in [is_NCpi0BDT(is_generic(x)) for x in samples]]
w_ = [y[y.reco_pi0_momentum < MAX].weight for y in [is_NCpi0BDT(is_generic(x)) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_pi0momentum, w_, BINS, (MIN,MAX), (0,300), colors, l_, r'Reco $\pi^{0}$ momentum [MeV/c]', 'Event counts')#, 
               #save_name='plot_open_dataset/ALL_kine_pi0_momentum.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/Prediction: %1.3f / %1.3f = %1.3f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_pi0costheta = [y.reco_pi0_costheta for y in [is_NCpi0BDT(is_generic(x)) for x in samples]]
w_ = [y.weight for y in [is_NCpi0BDT(is_generic(x)) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_pi0costheta, w_, 25, (-1,1), (0,120), colors, l_, r'Reco $\pi^{0}$ cos$\theta$', 'Event counts', 
               save_name='plot_open_dataset/ALL_kine_pi0_costheta.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/Prediction: %1.3f / %1.3f = %1.3f'%(sum_data, sum_pred, sum_data/sum_pred))

In [ ]:
hist_pi0mass = [y[(y.kine_pio_mass < 360) & (y.kine_pio_mass > 10)].kine_pio_mass for y in [is_FC(is_NCpi0BDT(is_generic(x))) for x in samples]]
w_ = [y[(y.kine_pio_mass < 360) & (y.kine_pio_mass > 10)].weight for y in [is_FC(is_NCpi0BDT(is_generic(x))) for x in samples]]
l_ = [x + ' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]
one_panel_plot(hist_pi0mass, w_, 30, (0,360), (0,74), colors, l_, r'Reco $\pi^{0}$ mass [MeV/c$^{2}$]', 'Event counts')#, 
               #save_name='plot_open_dataset/ALL_kine_pi0_mass.png')

sum_data = sum(w_[0])
sum_pred = sum([sum(x) for x in w_[1:]])
print('Data/Prediction: %1.3f / %1.3f = %1.3f'%(sum_data, sum_pred, sum_data/sum_pred))

### Ratio of true NCpi0 generated in Active Volume < 1500 GeV/c & < 4000 GeV/c

In [ ]:
ABC = sum(df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_pi0_momentum < 1500].weight)
XYZ = sum(df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_pi0_momentum < 4000].weight)
ABC/XYZ

### True and Reco distribution for pi0_momentum with binning

In [ ]:
BINS = [0, 100, 150, 200, 250, 300, 400, 500, 600, 800, 1000, 1500]

samples = [df_NCpi0inFV_efficiency, is_NCpi0BDT(df_NCpi0inFV)]
hist_true = [x[x.truth_pi0_momentum < BINS[-1]].truth_pi0_momentum for x in samples]
hist_reco = [y[y.reco_pi0_momentum < BINS[-1]].reco_pi0_momentum for y in samples]
w_true = [x[x.truth_pi0_momentum < BINS[-1]].weight for x in samples]
w_reco = [y[y.reco_pi0_momentum < BINS[-1]].weight for y in samples]

plt.figure(figsize=(14,5))
plt.subplot(121)
plt.hist(hist_true[0], bins=BINS, weights=w_true[0], label='True', color='blue', ls='--', histtype='step', lw=2)
plt.hist(hist_reco[0], bins=BINS, weights=w_reco[0], label='Reco', color='black', histtype='step', lw=2)
plt.xlabel(r'$P_{\pi^{0}}$ [MeV/c]')
plt.ylabel('Events')
plt.xlim(0,BINS[-1])
plt.ylim(0,160)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', title='Generated in TPC', prop={'size': legend_size})

plt.subplot(122)
plt.hist(hist_true[1], bins=BINS, weights=w_true[1], label='True', color='blue', ls='--', histtype='step', lw=2)
plt.hist(hist_reco[1], bins=BINS, weights=w_reco[1], label='Reco', color='black', histtype='step', lw=2)
plt.xlabel(r'$P_{\pi^{0}}$ [MeV/c]')
plt.xlim(0,BINS[-1])
plt.ylim(0,160)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', title='Selected with BDT', prop={'size': legend_size})

plt.tight_layout()
plt.savefig('plot_open_dataset/initial_dist.png')
plt.show()

### Cross Section final numbers (Enu)

In [ ]:
samples = [df_data, df_ext, df_dirt, df_outFV, df_badmatch, df_NCinFV, df_numuCCinFV, df_CCpi0inFV, df_NCpi0inFV[df_NCpi0inFV.truth_nuEnergy < 275]]
samples = [is_NCpi0BDT(is_generic(x[x.kine_reco_Enu < 8000])) for x in samples]
v_cv = [sum(x.weight) for x in samples]
v_err = [np.sqrt(sum(x.weight**2)) for x in samples]

# Target
count = 1.215e30
frac_stat = 0
frac_sys = 0.011
print('Target: %1.3e +- %1.3e +- %1.3e'%(count, count*frac_stat, count*frac_sys))
      
# POT
count = 5.32723e19
frac_stat = 0
frac_sys = 0.022
print('Target: %1.3e +- %1.3e +- %1.3e'%(count, count*frac_stat, count*frac_sys))

# BNB Data
count = v_cv[0]
frac_stat = v_err[0]/count
frac_sys = 0
print('BNB Data: %1.3f +- %1.3f +- %1.3f'%(count, count*frac_stat, count*frac_sys))

# EXTBNB Data
count = v_cv[1]
frac_stat = v_err[1]/count
frac_sys = 0
print('EXTBNB Data: %1.3f +- %1.3f +- %1.3f'%(count, count*frac_stat, count*frac_sys))

# Bkg MC
count = sum(v_cv[2:])
frac_stat = np.sqrt(sum([x**2 for x in v_err[2:]]))/count
frac_sys = 0
print('Bkg MC: %1.3f +- %1.3f +- %1.3f'%(count, count*frac_stat, count*frac_sys))

# Numerator MC
count = v_cv[0] - v_cv[1] - sum(v_cv[2:])
frac_stat = np.sqrt(v_err[0]**2 + v_err[1]**2 + np.sqrt(sum([x**2 for x in v_err[2:]]))**2)/count
frac_sys = 0
print('Numerator: %1.3f +- %1.3f +- %1.3f'%(count, count*frac_stat, count*frac_sys))

### Cross Section final number (Ppi0)

In [ ]:
samples = [df_data, df_ext, df_dirt, df_outFV, df_badmatch, df_NCinFV, df_numuCCinFV, df_CCpi0inFV, df_NCpi0inFV[df_NCpi0inFV.truth_pi0_momentum > 1500]]
samples = [is_NCpi0BDT(is_generic(x[x.reco_pi0_momentum < 3000])) for x in samples]
v_cv = [sum(x.weight) for x in samples]
v_err = [np.sqrt(sum(x.weight**2)) for x in samples]

# Target
count = 1.215e30
frac_stat = 0
frac_sys = 0.011
print('Target: %1.3e +- %1.3e +- %1.3e'%(count, count*frac_stat, count*frac_sys))
      
# POT
count = 5.32723e19
frac_stat = 0
frac_sys = 0.022
print('Target: %1.3e +- %1.3e +- %1.3e'%(count, count*frac_stat, count*frac_sys))

# BNB Data
count = v_cv[0]
frac_stat = v_err[0]/count
frac_sys = 0
print('BNB Data: %1.3f +- %1.3f +- %1.3f'%(count, count*frac_stat, count*frac_sys))

# EXTBNB Data
count = v_cv[1]
frac_stat = v_err[1]/count
frac_sys = 0
print('EXTBNB Data: %1.3f +- %1.3f +- %1.3f'%(count, count*frac_stat, count*frac_sys))

# Bkg MC
count = sum(v_cv[2:])
frac_stat = np.sqrt(sum([x**2 for x in v_err[2:]]))/count
frac_sys = 0
print('Bkg MC: %1.3f +- %1.3f +- %1.3f'%(count, count*frac_stat, count*frac_sys))

# Numerator MC
count = v_cv[0] - v_cv[1] - sum(v_cv[2:])
frac_stat = np.sqrt(v_err[0]**2 + v_err[1]**2 + np.sqrt(sum([x**2 for x in v_err[2:]]))**2)/count
frac_sys = 0
print('Numerator: %1.3f +- %1.3f +- %1.3f'%(count, count*frac_stat, count*frac_sys))

### Differential Cross Section final matrices

In [ ]:
file_wiener = '/home/gs627/wc_frame/LEEana/wiener_svd/wiener.root'
file_output = '/home/gs627/wc_frame/LEEana/wiener_svd/output.root'

In [ ]:
h2_R = uproot.open(file_wiener)['hR'].values
h2_Ac = uproot.open(file_output)['smear'].values
h1_Tbias = uproot.open(file_output)['bias'].values
mid = uproot.open(file_output)['bias'].edges[0:-1]
err = [(x-y)/2. for x,y in zip(uproot.open(file_output)['bias'].edges[1:], uproot.open(file_output)['bias'].edges[0:])]

h2_Cov = uproot.open(file_output)['unfcov'].values

BINS = [0, 100, 150, 200, 250, 300, 400, 500, 600, 800, 1000, 1500]

In [ ]:
plt.figure(figsize=(21,6))
plt.subplot(131)
plt.pcolormesh(h2_R, edgecolors='silver', cmap='PuRd', snap=True)
#plt.imshow(h2_R, origin='lower', aspect='auto')
plt.title('Matrix R', fontsize=15)
plt.xlabel('Truth bin index')
plt.ylabel('Reco bin index (FC, PC)')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.colorbar()

plt.subplot(132)
plt.pcolormesh(BINS, BINS, h2_Ac, edgecolors='silver', cmap='PuRd', snap=True)
#plt.imshow(h2_Ac, origin='lower', aspect='auto')
plt.title('Additional Smearing Matrix', fontsize=15)
plt.xlabel(r'$P^{\pi^{0}}_{true}$ [GeV/c]', fontsize=18)
plt.ylabel(r'$P^{\pi^{0}}_{true}$ [GeV/c]', fontsize=18)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.colorbar()

plt.subplot(133)
plt.errorbar(mid, h1_Tbias, xerr=err, yerr=0., color='red', fmt='none', ds='steps', lw=2)
plt.xlim(0,11)
plt.title('Intrinsic Bias', fontsize=15)
plt.xlabel('Truth bin index')
plt.ylabel('Relative to Model')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.tight_layout()
plt.savefig('plot_open_dataset/unfolding_matrix.png')
plt.show()

In [ ]:
def correlation_from_covariance(covariance):
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    correlation[covariance == 0] = 0
    return correlation

In [ ]:
plt.figure(figsize=(13,5))
plt.subplot(121)
plt.pcolormesh(BINS, BINS, h2_Cov, edgecolors='silver', cmap='PuRd', snap=True)
#plt.imshow(h2_Cov, origin='lower', aspect='auto')
plt.title('Unfolded Covariance', fontsize=13)
plt.xlabel(r'$P^{\pi^{0}}_{true}$ [GeV/c]', fontsize=15)
plt.ylabel(r'$P^{\pi^{0}}_{true}$ [GeV/c]', fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.colorbar()

plt.subplot(122)
plt.pcolormesh(BINS, BINS, correlation_from_covariance(h2_Cov), edgecolors='silver', cmap='PuRd', snap=True)
#plt.imshow(correlation_from_covariance(h2_Cov), origin='lower', aspect='auto')
plt.title('Correlation Matrix', fontsize=13)
plt.xlabel(r'$P^{\pi^{0}}_{true}$ [GeV/c]', fontsize=15)
#plt.ylabel(r'$P^{\pi^{0}}_{true}$ [GeV/c]', fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.colorbar()

plt.tight_layout()
plt.savefig('plot_open_dataset/unfolding_cov_corr.png')
plt.show()

## Pi0 Reconstruction Validation
### 2D plots - truth vs reco pio_energy_1 

In [ ]:
data = is_NCpi0BDT(is_generic(df_NCpi0inFV))
BINS = [0,50,100,150,200,300,400,600]
MAX = BINS[-1]

range_X = [0,MAX]
range_Y = [0,MAX]
weight_XY = data.weight
label_X = r'Truth energy [MeV]'
label_Y = r'Reco energy [MeV]'
hist_X = data.truth_pio_energy_1*1000.
hist_Y = data.kine_pio_energy_1

plt.figure(figsize=(7,5))
plt.hist2d(x=hist_X, y=hist_Y, weights=weight_XY, bins=[BINS,BINS], range=[range_X,range_Y], cmap='PuRd')
x = np.linspace(0,MAX,MAX)
plt.plot(x,x,c='red')
plt.xlabel(label_X)
plt.ylabel(label_Y)
plt.colorbar()
plt.tight_layout()
plt.savefig('plot_open_dataset/NCpi0BDT_TvR_pi0_energy_1.png')
plt.show()

### 2D plots - truth vs reco pio_energy_2

In [ ]:
data = is_NCpi0BDT(is_generic(df_NCpi0inFV))
BINS = [0,50,100,150,200,300,400,600]
MAX = BINS[-1]

range_X = [0,MAX]
range_Y = [0,MAX]
weight_XY = data.weight
label_X = r'Truth energy [MeV]'
label_Y = r'Reco energy [MeV]'
hist_X = data.truth_pio_energy_2*1000.
hist_Y = data.kine_pio_energy_2

plt.figure(figsize=(7,5))
plt.hist2d(x=hist_X, y=hist_Y, weights=weight_XY, bins=[BINS,BINS], range=[range_X,range_Y], cmap='PuRd')
x = np.linspace(0,MAX,MAX)
plt.plot(x,x,c='red')
plt.xlabel(label_X)
plt.ylabel(label_Y)
plt.colorbar()
plt.tight_layout()
plt.savefig('plot_open_dataset/NCpi0BDT_TvR_pi0_energy_2.png')
plt.show()

### 2D plots - truth vs reco pio_angle

In [ ]:
data = is_NCpi0BDT(is_generic(df_NCpi0inFV))
BINS = [0,15,30,45,60,90,120,150,180]
MAX = BINS[-1]

range_X = [0,MAX]
range_Y = [0,MAX]
weight_XY = data.weight
label_X = r'Truth Angle between showers [deg]'
label_Y = r'Reco Angle between showers [deg]'
hist_X = data.truth_pio_angle
hist_Y = data.kine_pio_angle

plt.figure(figsize=(7,5))
plt.hist2d(x=hist_X, y=hist_Y, weights=weight_XY, bins=[BINS,BINS], range=[range_X,range_Y], cmap='PuRd')
x = np.linspace(0,MAX,MAX)
plt.plot(x,x,c='red')
plt.xlabel(label_X)
plt.ylabel(label_Y)
plt.colorbar()
plt.tight_layout()
plt.savefig('plot_open_dataset/NCpi0BDT_TvR_pi0_ggAngle.png')
plt.show()

### 2D plots - truth vs reco pi0 momentum 

In [ ]:
data = is_NCpi0BDT(is_generic(df_NCpi0inFV))
BINS = [0, 100, 150, 200, 250, 300, 400, 500, 600, 800, 1000, 1500]
MAX = BINS[-1]

range_X = [0,MAX]
range_Y = [0,MAX]
weight_XY = data.weight
label_X = r'Truth $P_{\pi^{0}}$ [MeV/c]'
label_Y = r'Reco $P_{\pi^{0}}$ [MeV/c]' 
hist_X = data.truth_pi0_momentum
hist_Y = data.reco_pi0_momentum

plt.figure(figsize=(7,5))
plt.hist2d(x=hist_X, y=hist_Y, weights=weight_XY, bins=[BINS,BINS], range=[range_X,range_Y], cmap='PuRd')
x = np.linspace(0,MAX,MAX)
plt.plot(x,x,c='red')
plt.xlabel(label_X)
plt.ylabel(label_Y)
plt.colorbar()
plt.tight_layout()
plt.savefig('plot_open_dataset/NCpi0BDT_TvR_pi0_momentum.png')
plt.show()

### 2D plots - truth vs reco pi0 cos(theta)

In [ ]:
data = is_NCpi0BDT(is_generic(df_NCpi0inFV))
BINS = [-1.0, -0.62, -0.34, -0.13, 0.06, 0.2, 0.32, 0.42, 0.52, 0.6, 0.67, 0.73, 0.78, 0.83, 0.87, 0.91, 0.95, 0.975, 1.0]
MAX = BINS[-1]

range_X = [-1,MAX]
range_Y = [-1,MAX]
weight_XY = data.weight
label_X = r'Truth cos$\theta_{\pi^{0}}$'
label_Y = r'Reco cos$\theta_{\pi^{0}}$' 
hist_X = data.truth_pi0_costheta
hist_Y = data.reco_pi0_costheta

plt.figure(figsize=(7,5))
plt.hist2d(x=hist_X, y=hist_Y, weights=weight_XY, bins=[BINS,BINS], range=[range_X,range_Y], cmap='PuRd')
x = np.linspace(-1,MAX,1000)
plt.plot(x,x,c='red')
plt.xlabel(label_X)
plt.ylabel(label_Y)
plt.colorbar()
plt.tight_layout()
plt.savefig('plot_open_dataset/NCpi0BDT_TvR_pi0_costheta.png')
plt.show()

### 2D plots - truth vs reco Neutrino visible energy 

In [ ]:
data = is_NCpi0BDT(is_generic(df_NCpi0inFV))
BINS = [0, 200, 300, 400, 500, 600, 800, 1000, 1200, 1600]
MAX = BINS[-1]

range_X = [0,MAX]
range_Y = [0,MAX]
weight_XY = data.weight
label_X = r'Truth Energy Inside TPC [MeV]'
label_Y = r'Reco Energy Inside TPC [MeV]'
hist_X = data.truth_energyInside
hist_Y = data.kine_reco_Enu

plt.figure(figsize=(7,5))
plt.hist2d(x=hist_X, y=hist_Y, weights=weight_XY, bins=[BINS,BINS], range=[range_X,range_Y], cmap='PuRd')
x = np.linspace(0,MAX,MAX)
plt.plot(x,x,c='red')
plt.xlabel(label_X)
plt.ylabel(label_Y)
plt.colorbar()
plt.tight_layout()
plt.savefig('plot_open_dataset/NCpi0BDT_TvR_energyInside.png')
plt.show()

### 2D plots - truth vs reco Neutrino energy

In [ ]:
data = is_NCpi0BDT(is_generic(df_NCpi0inFV))
BINS = [0, 274, 400, 550, 700, 900, 1100, 1300, 1500, 1700, 2000, 3000, 4000]
MAX = BINS[-1]

range_X = [0,MAX]
range_Y = [0,MAX]
weight_XY = data.weight
label_X = r'Truth $E_{\nu}$ [MeV]'
label_Y = r'Reco Energy Inside TPC [MeV]'
hist_X = data.truth_nuEnergy
hist_Y = data.kine_reco_Enu

plt.figure(figsize=(7,5))
plt.hist2d(x=hist_X, y=hist_Y, weights=weight_XY, bins=[BINS,BINS], range=[range_X,range_Y], cmap='PuRd')
x = np.linspace(0,MAX,MAX)
plt.plot(x,x,c='red')
plt.xlabel(label_X)
plt.ylabel(label_Y)
plt.colorbar()
plt.tight_layout()
plt.savefig('plot_open_dataset/NCpi0BDT_TvR_nuEnergy.png')
plt.show()

### 2D plots - reco pi0 momentum vs reco pi0 cos(theta)

In [ ]:
bins_X = [0, 100, 150, 200, 250, 300, 400, 500, 600, 800, 1000, 1500]
bins_Y = [-1.0, -0.62, -0.34, -0.13, 0.06, 0.2, 0.32, 0.42, 0.52, 0.6, 0.67, 0.73, 0.78, 0.83, 0.87, 0.91, 0.95, 0.975, 1.0]
range_X = [0,bins_X[-1]]
range_Y = [-1,bins_Y[-1]]
label_X = r'Reco $P_{\pi^{0}}$ [MeV/c]'
label_Y = r'Reco cos$\theta_{\pi^{0}}$'

weight_1_XY = is_generic(df_NCpi0inFV).weight
hist_1_X = is_generic(df_NCpi0inFV).reco_pi0_momentum
hist_1_Y = is_generic(df_NCpi0inFV).reco_pi0_costheta

weight_2_XY = is_NCpi0BDT(is_generic(df_NCpi0inFV)).weight
hist_2_X = is_NCpi0BDT(is_generic(df_NCpi0inFV)).reco_pi0_momentum
hist_2_Y = is_NCpi0BDT(is_generic(df_NCpi0inFV)).reco_pi0_costheta

plt.figure(figsize=(14,5))
plt.subplot(121)
plt.hist2d(x=hist_1_Y, y=hist_1_X, weights=weight_1_XY, bins=[bins_Y,bins_X], range=[range_Y,range_X], cmap=plt.get_cmap('Blues'))
plt.xlabel(label_Y)
plt.ylabel(label_X)
plt.colorbar()
plt.subplot(122)
plt.hist2d(x=hist_2_Y, y=hist_2_X, weights=weight_2_XY, bins=[bins_Y,bins_X], range=[range_Y,range_X], cmap=plt.get_cmap('PuRd'))
plt.xlabel(label_Y)
#plt.ylabel(label_Y)
plt.colorbar()
plt.tight_layout()
plt.savefig('plot_open_dataset/NCpi0BDT_Reco_pi0_costheta_momentum.png')
plt.show()

In [ ]:
bins_X = [0, 100, 150, 200, 250, 300, 400, 500, 600, 800, 1000, 1500]
bins_Y = [-1.0, -0.62, -0.34, -0.13, 0.06, 0.2, 0.32, 0.42, 0.52, 0.6, 0.67, 0.73, 0.78, 0.83, 0.87, 0.91, 0.95, 0.975, 1.0]
range_X = [0,bins_X[-1]]
range_Y = [-1,bins_Y[-1]]
label_X = r'Truth $P_{\pi^{0}}$ [MeV/c]'
label_Y = r'Truth cos$\theta_{\pi^{0}}$'

weight_1_XY = is_generic(df_NCpi0inFV).weight
hist_1_X = is_generic(df_NCpi0inFV).truth_pi0_momentum
hist_1_Y = is_generic(df_NCpi0inFV).truth_pi0_costheta

weight_2_XY = is_NCpi0BDT(is_generic(df_NCpi0inFV)).weight
hist_2_X = is_NCpi0BDT(is_generic(df_NCpi0inFV)).truth_pi0_momentum
hist_2_Y = is_NCpi0BDT(is_generic(df_NCpi0inFV)).truth_pi0_costheta

plt.figure(figsize=(14,5))
plt.subplot(121)
plt.hist2d(x=hist_1_Y, y=hist_1_X, weights=weight_1_XY, bins=[bins_Y,bins_X], range=[range_Y,range_X], cmap=plt.get_cmap('Blues'))
plt.xlabel(label_Y)
plt.ylabel(label_X)
plt.colorbar()
plt.subplot(122)
plt.hist2d(x=hist_2_Y, y=hist_2_X, weights=weight_2_XY, bins=[bins_Y,bins_X], range=[range_Y,range_X], cmap=plt.get_cmap('PuRd'))
plt.xlabel(label_Y)
#plt.ylabel(label_Y)
plt.colorbar()
plt.tight_layout()
plt.savefig('plot_open_dataset/NCpi0BDT_Truth_pi0_costheta_momentum.png')
plt.show()

## Efficiency (truth_nuEnergy - single bin 0.275-8.0 GeV)

In [ ]:
'''
def sumw_X(X, wX, bins):
    sumw2 = []
    for left, right in zip(bins, bins[1:]): 
        ix = np.where((X >= left) & (X <= right))[0] 
        sumw2.append(np.sum(wX[ix] ** 2))
    return [np.sqrt(x) for x in sumw2]

def efficiency(N, D, wN, wD, bins):
    xcenter = [x+(y-x)/2. for x,y in zip(bins, bins[1:])]
    xerr = [(x-y)/2. for x,y in zip(bins[1:], bins)]
    
    hist_N, bins_N = np.histogram(N, weights=wN, bins=bins)
    hist_D, bins_D = np.histogram(D, weights=wD, bins=bins)
    ratio = [x/y if y > 0 else np.nan for x,y in zip(hist_N, hist_D)]
    
    sumw_N = sumw_X(N, np.asarray(wN), bins)
    sumw_D = sumw_X(D, np.asarray(wD), bins)
    yerr = []
    for a,b,c,d,e in zip(sumw_N, N, sumw_D, D, ratio):
        if e > 0: yerr.append(np.sqrt((a/d)**2 + ((c*b)/(d*d))**2))
        else: yerr.append(0)
            
    return xcenter, xerr, ratio, yerr
    
    
bins = [0, 274, 8000]
xrange = (bins[0],bins[-1])

N1 = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).truth_nuEnergy
N2 = is_NCpi0BDT(is_generic(df_NCpi0inFV)).truth_nuEnergy
D = df_NCpi0inFV_efficiency.truth_nuEnergy
                 
wN1 = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight
wN2 = is_NCpi0BDT(is_generic(df_NCpi0inFV)).weight
wD = df_NCpi0inFV_efficiency.weight   

xcenter_1, xerr_1, ratio_1, yerr_1 = efficiency(N1, D, wN1, wD, bins)
xcenter_2, xerr_2, ratio_2, yerr_2 = efficiency(N2, D, wN2, wD, bins)

plt.figure(figsize=(9,5))
plt.errorbar(xcenter_1, ratio_1, xerr=xerr_1, yerr=yerr_1, color='grey', label='N1', fmt='o')
plt.errorbar(xcenter_2, ratio_2, xerr=xerr_2, yerr=yerr_2, color='black', label='N2', fmt='o')
plt.ylim(0,1)
plt.xlim(xrange)
plt.show()

bins = [0, 274, 550, 700, 900, 1100, 1300, 1500, 1700, 2000, 3000, 4000, 8000]
xrange = (bins[0],bins[-1])

N1 = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).truth_nuEnergy
N2 = is_NCpi0BDT(is_generic(df_NCpi0inFV)).truth_nuEnergy
D = df_NCpi0inFV_efficiency.truth_nuEnergy
                 
wN1 = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight
wN2 = is_NCpi0BDT(is_generic(df_NCpi0inFV)).weight
wD = df_NCpi0inFV_efficiency.weight   

xcenter_1, xerr_1, ratio_1, yerr_1 = efficiency(N1, D, wN1, wD, bins)
xcenter_2, xerr_2, ratio_2, yerr_2 = efficiency(N2, D, wN2, wD, bins)

plt.figure(figsize=(9,5))
plt.errorbar(xcenter_1, ratio_1, xerr=xerr_1, yerr=yerr_1, color='grey', label='N1', fmt='o')
plt.errorbar(xcenter_2, ratio_2, xerr=xerr_2, yerr=yerr_2, color='black', label='N2', fmt='o')
plt.ylim(0,1)
plt.xlim(xrange)
plt.show()
'''

In [ ]:
bins = [0, 274, 8000]
xrange = (bins[0],bins[-1])


fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111)
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC', verbose=True)
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC', verbose=True)
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$E^{\nu}_{true}$ [MeV]')
ax.set_ylabel('Efficiency')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/eff_truth_nuEnergy_275_8000_1bin.png')
plt.show()

## Efficiency (truth_nuEnergy - many bins 0.275-8.0 GeV)

In [ ]:
bins = [0, 274, 400, 550, 700, 900, 1100, 1300, 1500, 1700, 2000, 3000, 4000, 8000]
xrange = (bins[0],bins[-1])

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111)
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$E^{\nu}_{true}$ [MeV]')
ax.set_ylabel('Efficiency')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/eff_truth_nuEnergy_275_8000_Nbin.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,5))
fig.add_subplot(121)
return_eff_plot(is_FC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_nuEnergy,
                is_FC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_nuEnergy)+list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_nuEnergy,
                is_FC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_nuEnergy)+list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best',  prop={'size': legend_size})
plt.xlabel(r'$E^{\nu}_{true}$ [MeV]')
plt.ylabel('Efficiency')



fig.add_subplot(122)
return_eff_plot(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy,
                is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC+PC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy,
                is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC+PC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best',  prop={'size': legend_size})
plt.xlabel(r'$E^{\nu}_{true}$ [MeV]')
plt.tight_layout()
plt.savefig('plot_open_dataset/eff_truth_nuEnergy_275_8000_Xp.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,5))
fig.add_subplot(121)
return_eff_plot(is_FC(is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_nuEnergy,
                is_FC(is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_nuEnergy)+list(is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_PC(is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_nuEnergy,
                is_FC(is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_nuEnergy)+list(is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_PC(is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best',  prop={'size': legend_size})
plt.xlabel(r'$E^{\nu}_{true}$ [MeV]')
plt.ylabel('Efficiency')



fig.add_subplot(122)
return_eff_plot(is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy,
                is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC+PC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy,
                is_fake_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_nuEnergy)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(is_fake_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC+PC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_nuEnergy)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_nuEnergy),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best',  prop={'size': legend_size})
plt.xlabel(r'$E^{\nu}_{true}$ [MeV]')
plt.tight_layout()
#plt.savefig('plot_open_dataset/eff_truth_nuEnergy_275_8000_Xp.png')
plt.show()

In [ ]:
NUM_P = 5

X = df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_nuEnergy < 800]
Y = is_NCpi0BDT(is_generic(X))
Y_0p = is_0p(Y)
Y_Np = is_Np(Y)

print(sum(Y_0p.weight))
print(sum(Y_Np.weight))


plt.figure(figsize=(14,5))
plt.suptitle('Truth nu energy < 800 MeV')
plt.subplot(121)
plt.hist(Y_Np.Num_total_protons, weights=Y_Np.weight, bins=NUM_P, range=(0,NUM_P), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.Num_total_protons, weights=Y_0p.weight, bins=NUM_P, range=(0,NUM_P), color='red', label='0p', alpha=0.5)
plt.xlim(0,NUM_P)
plt.xlabel('Total reco proton multiplicity')
plt.subplot(122)
plt.hist(Y_Np.Num_protons, weights=Y_Np.weight, bins=NUM_P, range=(0,NUM_P), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.Num_protons, weights=Y_0p.weight, bins=NUM_P, range=(0,NUM_P), color='red', label='0p', alpha=0.5)
plt.xlim(0,NUM_P)
plt.xlabel('Reco proton multiplicity (w/ threshold)')
plt.legend()
plt.show()


X = df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_nuEnergy > 800]
Y = is_NCpi0BDT(is_generic(X))
Y_0p = is_0p(Y)
Y_Np = is_Np(Y)
plt.figure(figsize=(14,5))
plt.suptitle('Truth nu energy > 800 MeV')
plt.subplot(121)
plt.hist(Y_Np.Num_total_protons, weights=Y_Np.weight, bins=NUM_P, range=(0,NUM_P), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.Num_total_protons, weights=Y_0p.weight, bins=NUM_P, range=(0,NUM_P), color='red', label='0p', alpha=0.5)
plt.xlim(0,NUM_P)
plt.xlabel('Total reco proton multiplicity')
plt.subplot(122)
plt.hist(Y_Np.Num_protons, weights=Y_Np.weight, bins=NUM_P, range=(0,NUM_P), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.Num_protons, weights=Y_0p.weight, bins=NUM_P, range=(0,NUM_P), color='red', label='0p', alpha=0.5)
plt.xlim(0,NUM_P)
plt.xlabel('Reco proton multiplicity (after threshold)')
plt.legend()
plt.show()

In [ ]:
NUM_P = 5

X = df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_pi0_costheta > 0.8]
Y = is_NCpi0BDT(is_generic(X))
Y_0p = is_0p(Y)
Y_Np = is_Np(Y)
plt.figure(figsize=(14,5))
plt.suptitle('Truth pi0 costheta > 0.8')
plt.subplot(121)
plt.hist(Y_Np.Num_total_protons, weights=Y_Np.weight, bins=NUM_P, range=(0,NUM_P), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.Num_total_protons, weights=Y_0p.weight, bins=NUM_P, range=(0,NUM_P), color='red', label='0p', alpha=0.5)
plt.xlim(0,NUM_P)
plt.xlabel('Total reco proton multiplicity')
plt.subplot(122)
plt.hist(Y_Np.Num_protons, weights=Y_Np.weight, bins=NUM_P, range=(0,NUM_P), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.Num_protons, weights=Y_0p.weight, bins=NUM_P, range=(0,NUM_P), color='red', label='0p', alpha=0.5)
plt.xlim(0,NUM_P)
plt.xlabel('Reco proton multiplicity (w/ threshold)')
plt.legend()
plt.show()


X = df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_pi0_costheta < 0.8]
Y = is_NCpi0BDT(is_generic(X))
Y_0p = is_0p(Y)
Y_Np = is_Np(Y)
plt.figure(figsize=(14,5))
plt.suptitle('Truth pi0 costheta < 0.8')
plt.subplot(121)
plt.hist(Y_Np.Num_total_protons, weights=Y_Np.weight, bins=NUM_P, range=(0,NUM_P), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.Num_total_protons, weights=Y_0p.weight, bins=NUM_P, range=(0,NUM_P), color='red', label='0p', alpha=0.5)
plt.xlim(0,NUM_P)
plt.xlabel('Total reco proton multiplicity')
plt.subplot(122)
plt.hist(Y_Np.Num_protons, weights=Y_Np.weight, bins=NUM_P, range=(0,NUM_P), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.Num_protons, weights=Y_0p.weight, bins=NUM_P, range=(0,NUM_P), color='red', label='0p', alpha=0.5)
plt.xlim(0,NUM_P)
plt.xlabel('Reco proton multiplicity (after threshold)')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
plt.title('Truth pi0 costheta < 0.8')
X = df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_pi0_costheta < 0.8]
Y = is_NCpi0BDT(is_generic(X))
Y_0p = is_0p(Y)
Y_Np = is_Np(Y)
plt.hist(Y_Np.truth_pio_energy_1*1000, weights=Y_Np.weight, bins=20, range=(0,1000), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.truth_pio_energy_1*1000, weights=Y_0p.weight, bins=20, range=(0,1000), color='red', label='0p', alpha=0.5)
plt.xlim(0,1000)

plt.subplot(122)
plt.title('Truth pi0 costheta > 0.8')
X = df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_pi0_costheta > 0.8]
Y = is_NCpi0BDT(is_generic(X))
Y_0p = is_0p(Y)
Y_Np = is_Np(Y)
plt.hist(Y_Np.truth_pio_energy_1*1000, weights=Y_Np.weight, bins=20, range=(0,1000), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.truth_pio_energy_1*1000, weights=Y_0p.weight, bins=20, range=(0,1000), color='red', label='0p', alpha=0.5)
plt.xlim(0,1000)

plt.legend()
plt.show()



plt.figure(figsize=(14,5))
plt.subplot(121)
plt.title('Truth pi0 costheta < 0.8')
X = df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_pi0_costheta < 0.8]
Y = is_NCpi0BDT(is_generic(X))
Y_0p = is_0p(Y)
Y_Np = is_Np(Y)
plt.hist(Y_Np.truth_pi0_momentum, weights=Y_Np.weight, bins=20, range=(0,1000), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.truth_pi0_momentum, weights=Y_0p.weight, bins=20, range=(0,1000), color='red', label='0p', alpha=0.5)
plt.xlim(0,1000)

plt.subplot(122)
plt.title('Truth pi0 costheta > 0.8')
X = df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_pi0_costheta > 0.8]
Y = is_NCpi0BDT(is_generic(X))
Y_0p = is_0p(Y)
Y_Np = is_Np(Y)
plt.hist(Y_Np.truth_pi0_momentum, weights=Y_Np.weight, bins=20, range=(0,1000), color='blue', label='Np', alpha=0.5)
plt.hist(Y_0p.truth_pi0_momentum, weights=Y_0p.weight, bins=20, range=(0,1000), color='red', label='0p', alpha=0.5)
plt.xlim(0,1000)

plt.legend()
plt.show()

In [ ]:
bins = np.linspace(0,500,25)
xrange = (bins[0],bins[-1])

X = df_NCpi0inFV_efficiency[df_NCpi0inFV_efficiency.truth_nuEnergy < 800]



fig = plt.figure(figsize=(14,5))
fig.add_subplot(121)
return_eff_plot(is_FC(is_0p(is_NCpi0BDT(is_generic(X)))).truth_pio_energy_1*1000.,
                is_FC(is_0p(is_NCpi0BDT(is_generic(X)))).weight,
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(X)))).truth_pio_energy_1*1000.)+list(is_Np(is_NCpi0BDT(is_generic(X))).truth_pio_energy_1*1000.)+list(isnot_NCpi0BDT(is_generic(X)).truth_pio_energy_1*1000.)+list(isnot_generic(X).truth_pio_energy_1*1000.),
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(X)))).weight)+list(is_Np(is_NCpi0BDT(is_generic(X))).weight)+list(isnot_NCpi0BDT(is_generic(X)).weight)+list(isnot_generic(X).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_Np(is_NCpi0BDT(is_generic(X)))).truth_pio_energy_1*1000.,
                is_FC(is_Np(is_NCpi0BDT(is_generic(X)))).weight,
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(X)))).truth_pio_energy_1*1000.)+list(is_0p(is_NCpi0BDT(is_generic(X))).truth_pio_energy_1*1000.)+list(isnot_NCpi0BDT(is_generic(X)).truth_pio_energy_1*1000.)+list(isnot_generic(X).truth_pio_energy_1*1000.),
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(X)))).weight)+list(is_0p(is_NCpi0BDT(is_generic(X))).weight)+list(isnot_NCpi0BDT(is_generic(X)).weight)+list(isnot_generic(X).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(X))).truth_pio_energy_1*1000.,
                is_FC(is_NCpi0BDT(is_generic(X))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(X))).truth_pio_energy_1*1000.)+list(isnot_NCpi0BDT(is_generic(X)).truth_pio_energy_1*1000.)+list(isnot_generic(X).truth_pio_energy_1*1000.),
                list(is_PC(is_NCpi0BDT(is_generic(X))).weight)+list(isnot_NCpi0BDT(is_generic(X)).weight)+list(isnot_generic(X).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,0.5)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best',  prop={'size': legend_size})
plt.xlabel(r'$E^{\gamma}_{true}$ [MeV]')
plt.ylabel('Efficiency')



fig.add_subplot(122)
return_eff_plot(is_0p(is_NCpi0BDT(is_generic(X))).truth_pio_energy_1*1000.,
                is_0p(is_NCpi0BDT(is_generic(X))).weight,
                list(is_Np(is_NCpi0BDT(is_generic(X))).truth_pio_energy_1*1000.)+list(isnot_NCpi0BDT(is_generic(X)).truth_pio_energy_1*1000.)+list(isnot_generic(X).truth_pio_energy_1*1000.),
                list(is_Np(is_NCpi0BDT(is_generic(X))).weight)+list(isnot_NCpi0BDT(is_generic(X)).weight)+list(isnot_generic(X).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC+PC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_Np(is_NCpi0BDT(is_generic(X))).truth_pio_energy_1*1000.,
                is_Np(is_NCpi0BDT(is_generic(X))).weight,
                list(is_0p(is_NCpi0BDT(is_generic(X))).truth_pio_energy_1*1000.)+list(isnot_NCpi0BDT(is_generic(X)).truth_pio_energy_1*1000.)+list(isnot_generic(X).truth_pio_energy_1*1000.),
                list(is_0p(is_NCpi0BDT(is_generic(X))).weight)+list(isnot_NCpi0BDT(is_generic(X)).weight)+list(isnot_generic(X).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC+PC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(X)).truth_pio_energy_1*1000.,
                is_NCpi0BDT(is_generic(X)).weight,
                list(isnot_NCpi0BDT(is_generic(X)).truth_pio_energy_1*1000.)+list(isnot_generic(X).truth_pio_energy_1*1000.),
                list(isnot_NCpi0BDT(is_generic(X)).weight)+list(isnot_generic(X).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,0.5)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best',  prop={'size': legend_size})
plt.xlabel(r'$E^{\gamma}_{true}$ [MeV]')
plt.tight_layout()
#plt.savefig('plot_open_dataset/eff_truth_leaning_gamma_Xp.png')
plt.show()

In [ ]:
bins = np.linspace(0,500,25)
xrange = (bins[0],bins[-1])

fig = plt.figure(figsize=(14,5))
fig.add_subplot(121)
return_eff_plot(is_FC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pio_energy_2*1000.,
                is_FC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pio_energy_2*1000.)+list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pio_energy_2*1000.)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pio_energy_2*1000.)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pio_energy_2*1000.),
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pio_energy_2*1000.,
                is_FC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pio_energy_2*1000.)+list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pio_energy_2*1000.)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pio_energy_2*1000.)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pio_energy_2*1000.),
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pio_energy_2*1000.,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pio_energy_2*1000.)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pio_energy_2*1000.)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pio_energy_2*1000.),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,0.5)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best',  prop={'size': legend_size})
plt.xlabel(r'$E^{\nu}_{true}$ [MeV]')
plt.ylabel('Efficiency')



fig.add_subplot(122)
return_eff_plot(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pio_energy_2*1000.,
                is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pio_energy_2*1000.)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pio_energy_2*1000.)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pio_energy_2*1000.),
                list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC+PC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pio_energy_2*1000.,
                is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pio_energy_2*1000.)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pio_energy_2*1000.)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pio_energy_2*1000.),
                list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC+PC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pio_energy_2*1000.,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pio_energy_2*1000.)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pio_energy_2*1000.),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,0.5)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best',  prop={'size': legend_size})
plt.xlabel(r'$E^{\nu}_{true}$ [MeV]')
plt.tight_layout()
#plt.savefig('plot_open_dataset/eff_truth_pio_energy_2*1000._275_8000_Xp.png')
plt.show()

## Efficiency (truth_pi0_momentum)

In [ ]:
bins = [0, 1500]
xrange = (bins[0],bins[-1])

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111)
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$P^{\pi^{0}}_{true}$ [MeV/c]')
ax.set_ylabel('Efficiency')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/eff_truth_pi0_momentum_0_1500_1bin.png')
plt.show()

In [ ]:
bins = [0, 100, 150, 200, 250, 300, 400, 500, 600, 800, 1000, 1500]
xrange = (bins[0],bins[-1])

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111)
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$P^{\pi^{0}}_{true}$ [MeV/c]')
ax.set_ylabel('Efficiency')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/eff_truth_pi0_momentum_0_1500_Nbin.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,5))
fig.add_subplot(121)
return_eff_plot(is_FC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pi0_momentum,
                is_FC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pi0_momentum)+list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pi0_momentum,
                is_FC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pi0_momentum)+list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$P^{\pi^{0}}_{true}$ [MeV/c]')
plt.ylabel('Efficiency')



fig.add_subplot(122)
return_eff_plot(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum,
                is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC+PC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum,
                is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_momentum)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC+PC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_momentum)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_momentum),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$P^{\pi^{0}}_{true}$ [MeV/c]')
plt.tight_layout()
plt.savefig('plot_open_dataset/eff_truth_pi0_momentum_0_1500_Xp.png')
plt.show()

## Efficiency (truth_pi0_costheta)

In [ ]:
bins = [-1.0, -0.62, -0.34, -0.13, 0.06, 0.2, 0.32, 0.42, 0.52, 0.6, 0.67, 0.73, 0.78, 0.83, 0.87, 0.91, 0.95, 0.975, 1.0]
xrange = (bins[0],bins[-1])

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111)
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_costheta),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_costheta),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$cos\theta^{\pi^{0}}_{true}$')
ax.set_ylabel('Efficiency')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/eff_truth_pi0_costheta_Nbin.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,5))
fig.add_subplot(121)
return_eff_plot(is_FC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pi0_costheta,
                is_FC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pi0_costheta)+list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_costheta),
                list(is_PC(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pi0_costheta,
                is_FC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight,
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).truth_pi0_costheta)+list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_costheta),
                list(is_PC(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)))).weight)+list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta,
                is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_costheta),
                list(is_PC(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='darkgrey', fmt='o', label=r'FC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$cos\theta^{\pi^{0}}_{true}$')
plt.ylabel('Efficiency')



fig.add_subplot(122)
return_eff_plot(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta,
                is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_costheta),
                list(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightcoral', fmt='^', label=r'FC+PC 0p')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta,
                is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight,
                list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).truth_pi0_costheta)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_costheta),
                list(is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency))).weight)+list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='lightskyblue', fmt='v', label=r'FC+PC Np')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta,
                is_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight,
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).truth_pi0_costheta)+list(isnot_generic(df_NCpi0inFV_efficiency).truth_pi0_costheta),
                list(isnot_NCpi0BDT(is_generic(df_NCpi0inFV_efficiency)).weight)+list(isnot_generic(df_NCpi0inFV_efficiency).weight),
                bins, xrange, color='black', fmt='o', label=r'FC+PC (0+N)p')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$cos\theta^{\pi^{0}}_{true}$')
plt.tight_layout()
plt.savefig('plot_open_dataset/eff_truth_pi0_costheta_Xp.png')
plt.show()

## Purity (kine_reco_Enu)

In [ ]:
bins = [0, 274, 400, 550, 700, 900, 1100, 1300, 1500, 1700, 2000]
xrange = (bins[0],bins[-1])

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111)
hist = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).kine_reco_Enu
weight = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight

hist_ext = is_FC(is_NCpi0BDT(is_generic(df_ext))).kine_reco_Enu
weight_ext = is_FC(is_NCpi0BDT(is_generic(df_ext))).weight

hist_over = [is_FC(is_NCpi0BDT(is_generic(df_NCinFV))).kine_reco_Enu,
             is_FC(is_NCpi0BDT(is_generic(df_outFV))).kine_reco_Enu,
             is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV))).kine_reco_Enu,
             is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV))).kine_reco_Enu,
             is_FC(is_NCpi0BDT(is_generic(df_badmatch))).kine_reco_Enu]
weight_over = [is_FC(is_NCpi0BDT(is_generic(df_NCinFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_outFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_badmatch))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV))).kine_reco_Enu]
weight_int = [is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV))).weight]

hist_dirt = is_FC(is_NCpi0BDT(is_generic(df_dirt))).kine_reco_Enu
weight_dirt = is_FC(is_NCpi0BDT(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='darkgrey', fmt='o', label=r'FC')
# -------------------------------------------------------------------------------------------------------------------
hist = is_NCpi0BDT(is_generic(df_NCpi0inFV)).kine_reco_Enu
weight = is_NCpi0BDT(is_generic(df_NCpi0inFV)).weight

hist_ext = is_NCpi0BDT(is_generic(df_ext)).kine_reco_Enu
weight_ext = is_NCpi0BDT(is_generic(df_ext)).weight

hist_over = [is_NCpi0BDT(is_generic(df_NCinFV)).kine_reco_Enu,
             is_NCpi0BDT(is_generic(df_outFV)).kine_reco_Enu,
             is_NCpi0BDT(is_generic(df_CCpi0inFV)).kine_reco_Enu,
             is_NCpi0BDT(is_generic(df_numuCCinFV)).kine_reco_Enu,
             is_NCpi0BDT(is_generic(df_badmatch)).kine_reco_Enu]
weight_over = [is_NCpi0BDT(is_generic(df_NCinFV)).weight,
               is_NCpi0BDT(is_generic(df_outFV)).weight,
               is_NCpi0BDT(is_generic(df_CCpi0inFV)).weight,
               is_NCpi0BDT(is_generic(df_numuCCinFV)).weight,
               is_NCpi0BDT(is_generic(df_badmatch)).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_NCpi0BDT(is_generic(df_nueCCinFV)).kine_reco_Enu]
weight_int = [is_NCpi0BDT(is_generic(df_nueCCinFV)).weight]

hist_dirt = is_NCpi0BDT(is_generic(df_dirt)).kine_reco_Enu
weight_dirt = is_NCpi0BDT(is_generic(df_dirt)).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='black', fmt='o', label=r'FC+PC')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$E^{\nu}_{reco}$ [MeV]')
ax.set_ylabel('Purity')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/pur_kine_reco_Enu_Nbin.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,5))
fig.add_subplot(121)
hist = is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).kine_reco_Enu
weight = is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).weight

hist_ext = is_0p(is_FC(is_NCpi0BDT(is_generic(df_ext)))).kine_reco_Enu
weight_ext = is_0p(is_FC(is_NCpi0BDT(is_generic(df_ext)))).weight

hist_over = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).kine_reco_Enu,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).kine_reco_Enu,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).kine_reco_Enu,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).kine_reco_Enu,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).kine_reco_Enu]
weight_over = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).kine_reco_Enu]
weight_int = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).weight]

hist_dirt = is_0p(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).kine_reco_Enu
weight_dirt = is_0p(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='lightcoral', fmt='o', label=r'FC 0p')
# -------------------------------------------------------------------------------------------------------------------
hist = is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).kine_reco_Enu
weight = is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).weight

hist_ext = is_Np(is_FC(is_NCpi0BDT(is_generic(df_ext)))).kine_reco_Enu
weight_ext = is_Np(is_FC(is_NCpi0BDT(is_generic(df_ext)))).weight

hist_over = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).kine_reco_Enu,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).kine_reco_Enu,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).kine_reco_Enu,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).kine_reco_Enu,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).kine_reco_Enu]
weight_over = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).kine_reco_Enu]
weight_int = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).weight]

hist_dirt = is_Np(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).kine_reco_Enu
weight_dirt = is_Np(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='deepskyblue', fmt='o', label=r'FC Np')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$E^{\nu}_{reco}$ [MeV]')
plt.ylabel('Purity')



fig.add_subplot(122)
hist = is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV))).kine_reco_Enu
weight = is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight

hist_ext = is_0p(is_NCpi0BDT(is_generic(df_ext))).kine_reco_Enu
weight_ext = is_0p(is_NCpi0BDT(is_generic(df_ext))).weight

hist_over = [is_0p(is_NCpi0BDT(is_generic(df_NCinFV))).kine_reco_Enu,
             is_0p(is_NCpi0BDT(is_generic(df_outFV))).kine_reco_Enu,
             is_0p(is_NCpi0BDT(is_generic(df_CCpi0inFV))).kine_reco_Enu,
             is_0p(is_NCpi0BDT(is_generic(df_numuCCinFV))).kine_reco_Enu,
             is_0p(is_NCpi0BDT(is_generic(df_badmatch))).kine_reco_Enu]
weight_over = [is_0p(is_NCpi0BDT(is_generic(df_NCinFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_outFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_CCpi0inFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_numuCCinFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_badmatch))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_0p(is_NCpi0BDT(is_generic(df_nueCCinFV))).kine_reco_Enu]
weight_int = [is_0p(is_NCpi0BDT(is_generic(df_nueCCinFV))).weight]

hist_dirt = is_0p(is_NCpi0BDT(is_generic(df_dirt))).kine_reco_Enu
weight_dirt = is_0p(is_NCpi0BDT(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='lightcoral', fmt='o', label=r'FC+PC 0p')
# -------------------------------------------------------------------------------------------------------------------
hist = is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV))).kine_reco_Enu
weight = is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight

hist_ext = is_Np(is_NCpi0BDT(is_generic(df_ext))).kine_reco_Enu
weight_ext = is_Np(is_NCpi0BDT(is_generic(df_ext))).weight

hist_over = [is_Np(is_NCpi0BDT(is_generic(df_NCinFV))).kine_reco_Enu,
             is_Np(is_NCpi0BDT(is_generic(df_outFV))).kine_reco_Enu,
             is_Np(is_NCpi0BDT(is_generic(df_CCpi0inFV))).kine_reco_Enu,
             is_Np(is_NCpi0BDT(is_generic(df_numuCCinFV))).kine_reco_Enu,
             is_Np(is_NCpi0BDT(is_generic(df_badmatch))).kine_reco_Enu]
weight_over = [is_Np(is_NCpi0BDT(is_generic(df_NCinFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_outFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_CCpi0inFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_numuCCinFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_badmatch))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_Np(is_NCpi0BDT(is_generic(df_nueCCinFV))).kine_reco_Enu]
weight_int = [is_Np(is_NCpi0BDT(is_generic(df_nueCCinFV))).weight]

hist_dirt = is_Np(is_NCpi0BDT(is_generic(df_dirt))).kine_reco_Enu
weight_dirt = is_Np(is_NCpi0BDT(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='deepskyblue', fmt='o', label=r'FC+PC Np')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$E^{\nu}_{reco}$ [MeV]')
plt.tight_layout()
plt.savefig('plot_open_dataset/pur_kine_reco_Enu_Xp.png')
plt.show()

## Purity (reco_pi0_momentum)

In [ ]:
bins = [0, 100, 150, 200, 250, 300, 400, 500, 600, 800, 1000, 1500, 2000, 3000]
xrange = (bins[0],bins[-1])

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111)
hist = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).reco_pi0_momentum
weight = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight

hist_ext = is_FC(is_NCpi0BDT(is_generic(df_ext))).reco_pi0_momentum
weight_ext = is_FC(is_NCpi0BDT(is_generic(df_ext))).weight

hist_over = [is_FC(is_NCpi0BDT(is_generic(df_NCinFV))).reco_pi0_momentum,
             is_FC(is_NCpi0BDT(is_generic(df_outFV))).reco_pi0_momentum,
             is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV))).reco_pi0_momentum,
             is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV))).reco_pi0_momentum,
             is_FC(is_NCpi0BDT(is_generic(df_badmatch))).reco_pi0_momentum]
weight_over = [is_FC(is_NCpi0BDT(is_generic(df_NCinFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_outFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_badmatch))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV))).reco_pi0_momentum]
weight_int = [is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV))).weight]

hist_dirt = is_FC(is_NCpi0BDT(is_generic(df_dirt))).reco_pi0_momentum
weight_dirt = is_FC(is_NCpi0BDT(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='darkgrey', fmt='o', label=r'FC')
# -------------------------------------------------------------------------------------------------------------------
hist = is_NCpi0BDT(is_generic(df_NCpi0inFV)).reco_pi0_momentum
weight = is_NCpi0BDT(is_generic(df_NCpi0inFV)).weight

hist_ext = is_NCpi0BDT(is_generic(df_ext)).reco_pi0_momentum
weight_ext = is_NCpi0BDT(is_generic(df_ext)).weight

hist_over = [is_NCpi0BDT(is_generic(df_NCinFV)).reco_pi0_momentum,
             is_NCpi0BDT(is_generic(df_outFV)).reco_pi0_momentum,
             is_NCpi0BDT(is_generic(df_CCpi0inFV)).reco_pi0_momentum,
             is_NCpi0BDT(is_generic(df_numuCCinFV)).reco_pi0_momentum,
             is_NCpi0BDT(is_generic(df_badmatch)).reco_pi0_momentum]
weight_over = [is_NCpi0BDT(is_generic(df_NCinFV)).weight,
               is_NCpi0BDT(is_generic(df_outFV)).weight,
               is_NCpi0BDT(is_generic(df_CCpi0inFV)).weight,
               is_NCpi0BDT(is_generic(df_numuCCinFV)).weight,
               is_NCpi0BDT(is_generic(df_badmatch)).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_NCpi0BDT(is_generic(df_nueCCinFV)).reco_pi0_momentum]
weight_int = [is_NCpi0BDT(is_generic(df_nueCCinFV)).weight]

hist_dirt = is_NCpi0BDT(is_generic(df_dirt)).reco_pi0_momentum
weight_dirt = is_NCpi0BDT(is_generic(df_dirt)).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='black', fmt='o', label=r'FC+PC')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$P^{\pi^{0}}_{reco}$ [MeV/c]')
ax.set_ylabel('Purity')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/pur_reco_pi0_momentum_Nbin.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,5))
fig.add_subplot(121)
hist = is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).reco_pi0_momentum
weight = is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).weight

hist_ext = is_0p(is_FC(is_NCpi0BDT(is_generic(df_ext)))).reco_pi0_momentum
weight_ext = is_0p(is_FC(is_NCpi0BDT(is_generic(df_ext)))).weight

hist_over = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).reco_pi0_momentum,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).reco_pi0_momentum,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).reco_pi0_momentum,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).reco_pi0_momentum,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).reco_pi0_momentum]
weight_over = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).reco_pi0_momentum]
weight_int = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).weight]

hist_dirt = is_0p(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).reco_pi0_momentum
weight_dirt = is_0p(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='lightcoral', fmt='o', label=r'FC 0p')
# -------------------------------------------------------------------------------------------------------------------
hist = is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).reco_pi0_momentum
weight = is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).weight

hist_ext = is_Np(is_FC(is_NCpi0BDT(is_generic(df_ext)))).reco_pi0_momentum
weight_ext = is_Np(is_FC(is_NCpi0BDT(is_generic(df_ext)))).weight

hist_over = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).reco_pi0_momentum,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).reco_pi0_momentum,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).reco_pi0_momentum,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).reco_pi0_momentum,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).reco_pi0_momentum]
weight_over = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).reco_pi0_momentum]
weight_int = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).weight]

hist_dirt = is_Np(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).reco_pi0_momentum
weight_dirt = is_Np(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='deepskyblue', fmt='o', label=r'FC Np')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$P^{\pi^{0}}_{reco}$ [MeV/c]')
plt.ylabel('Purity')



fig.add_subplot(122)
hist = is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV))).reco_pi0_momentum
weight = is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight

hist_ext = is_0p(is_NCpi0BDT(is_generic(df_ext))).reco_pi0_momentum
weight_ext = is_0p(is_NCpi0BDT(is_generic(df_ext))).weight

hist_over = [is_0p(is_NCpi0BDT(is_generic(df_NCinFV))).reco_pi0_momentum,
             is_0p(is_NCpi0BDT(is_generic(df_outFV))).reco_pi0_momentum,
             is_0p(is_NCpi0BDT(is_generic(df_CCpi0inFV))).reco_pi0_momentum,
             is_0p(is_NCpi0BDT(is_generic(df_numuCCinFV))).reco_pi0_momentum,
             is_0p(is_NCpi0BDT(is_generic(df_badmatch))).reco_pi0_momentum]
weight_over = [is_0p(is_NCpi0BDT(is_generic(df_NCinFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_outFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_CCpi0inFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_numuCCinFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_badmatch))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_0p(is_NCpi0BDT(is_generic(df_nueCCinFV))).reco_pi0_momentum]
weight_int = [is_0p(is_NCpi0BDT(is_generic(df_nueCCinFV))).weight]

hist_dirt = is_0p(is_NCpi0BDT(is_generic(df_dirt))).reco_pi0_momentum
weight_dirt = is_0p(is_NCpi0BDT(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='lightcoral', fmt='o', label=r'FC+PC 0p')
# -------------------------------------------------------------------------------------------------------------------
hist = is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV))).reco_pi0_momentum
weight = is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight

hist_ext = is_Np(is_NCpi0BDT(is_generic(df_ext))).reco_pi0_momentum
weight_ext = is_Np(is_NCpi0BDT(is_generic(df_ext))).weight

hist_over = [is_Np(is_NCpi0BDT(is_generic(df_NCinFV))).reco_pi0_momentum,
             is_Np(is_NCpi0BDT(is_generic(df_outFV))).reco_pi0_momentum,
             is_Np(is_NCpi0BDT(is_generic(df_CCpi0inFV))).reco_pi0_momentum,
             is_Np(is_NCpi0BDT(is_generic(df_numuCCinFV))).reco_pi0_momentum,
             is_Np(is_NCpi0BDT(is_generic(df_badmatch))).reco_pi0_momentum]
weight_over = [is_Np(is_NCpi0BDT(is_generic(df_NCinFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_outFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_CCpi0inFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_numuCCinFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_badmatch))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_Np(is_NCpi0BDT(is_generic(df_nueCCinFV))).reco_pi0_momentum]
weight_int = [is_Np(is_NCpi0BDT(is_generic(df_nueCCinFV))).weight]

hist_dirt = is_Np(is_NCpi0BDT(is_generic(df_dirt))).reco_pi0_momentum
weight_dirt = is_Np(is_NCpi0BDT(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='deepskyblue', fmt='o', label=r'FC+PC Np')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$P^{\pi^{0}}_{reco}$ [MeV/c]')
plt.tight_layout()
plt.savefig('plot_open_dataset/pur_reco_pi0_momentum_Xp.png')
plt.show()

## Purity (reco_pi0_costheta)

In [ ]:
bins = [-1.0, -0.62, -0.34, -0.13, 0.06, 0.2, 0.32, 0.42, 0.52, 0.6, 0.67, 0.73, 0.78, 0.83, 0.87, 0.91, 0.95, 0.975, 1.0]
xrange = (bins[0],bins[-1])

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111)
hist = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).reco_pi0_costheta
weight = is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight

hist_ext = is_FC(is_NCpi0BDT(is_generic(df_ext))).reco_pi0_costheta
weight_ext = is_FC(is_NCpi0BDT(is_generic(df_ext))).weight

hist_over = [is_FC(is_NCpi0BDT(is_generic(df_NCinFV))).reco_pi0_costheta,
             is_FC(is_NCpi0BDT(is_generic(df_outFV))).reco_pi0_costheta,
             is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV))).reco_pi0_costheta,
             is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV))).reco_pi0_costheta,
             is_FC(is_NCpi0BDT(is_generic(df_badmatch))).reco_pi0_costheta]
weight_over = [is_FC(is_NCpi0BDT(is_generic(df_NCinFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_outFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV))).weight,
               is_FC(is_NCpi0BDT(is_generic(df_badmatch))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV))).reco_pi0_costheta]
weight_int = [is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV))).weight]

hist_dirt = is_FC(is_NCpi0BDT(is_generic(df_dirt))).reco_pi0_costheta
weight_dirt = is_FC(is_NCpi0BDT(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='darkgrey', fmt='o', label=r'FC')
# -------------------------------------------------------------------------------------------------------------------
hist = is_NCpi0BDT(is_generic(df_NCpi0inFV)).reco_pi0_costheta
weight = is_NCpi0BDT(is_generic(df_NCpi0inFV)).weight

hist_ext = is_NCpi0BDT(is_generic(df_ext)).reco_pi0_costheta
weight_ext = is_NCpi0BDT(is_generic(df_ext)).weight

hist_over = [is_NCpi0BDT(is_generic(df_NCinFV)).reco_pi0_costheta,
             is_NCpi0BDT(is_generic(df_outFV)).reco_pi0_costheta,
             is_NCpi0BDT(is_generic(df_CCpi0inFV)).reco_pi0_costheta,
             is_NCpi0BDT(is_generic(df_numuCCinFV)).reco_pi0_costheta,
             is_NCpi0BDT(is_generic(df_badmatch)).reco_pi0_costheta]
weight_over = [is_NCpi0BDT(is_generic(df_NCinFV)).weight,
               is_NCpi0BDT(is_generic(df_outFV)).weight,
               is_NCpi0BDT(is_generic(df_CCpi0inFV)).weight,
               is_NCpi0BDT(is_generic(df_numuCCinFV)).weight,
               is_NCpi0BDT(is_generic(df_badmatch)).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_NCpi0BDT(is_generic(df_nueCCinFV)).reco_pi0_costheta]
weight_int = [is_NCpi0BDT(is_generic(df_nueCCinFV)).weight]

hist_dirt = is_NCpi0BDT(is_generic(df_dirt)).reco_pi0_costheta
weight_dirt = is_NCpi0BDT(is_generic(df_dirt)).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='black', fmt='o', label=r'FC+PC')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$cos\theta^{\pi^{0}}_{reco}$')
ax.set_ylabel('Purity')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/pur_reco_pi0_costheta_Nbin.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,5))
fig.add_subplot(121)
hist = is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).reco_pi0_costheta
weight = is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).weight

hist_ext = is_0p(is_FC(is_NCpi0BDT(is_generic(df_ext)))).reco_pi0_costheta
weight_ext = is_0p(is_FC(is_NCpi0BDT(is_generic(df_ext)))).weight

hist_over = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).reco_pi0_costheta,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).reco_pi0_costheta,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).reco_pi0_costheta,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).reco_pi0_costheta,
             is_0p(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).reco_pi0_costheta]
weight_over = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).weight,
               is_0p(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).reco_pi0_costheta]
weight_int = [is_0p(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).weight]

hist_dirt = is_0p(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).reco_pi0_costheta
weight_dirt = is_0p(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='lightcoral', fmt='o', label=r'FC 0p')
# -------------------------------------------------------------------------------------------------------------------
hist = is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).reco_pi0_costheta
weight = is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCpi0inFV)))).weight

hist_ext = is_Np(is_FC(is_NCpi0BDT(is_generic(df_ext)))).reco_pi0_costheta
weight_ext = is_Np(is_FC(is_NCpi0BDT(is_generic(df_ext)))).weight

hist_over = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).reco_pi0_costheta,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).reco_pi0_costheta,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).reco_pi0_costheta,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).reco_pi0_costheta,
             is_Np(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).reco_pi0_costheta]
weight_over = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_NCinFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_outFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_CCpi0inFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_numuCCinFV)))).weight,
               is_Np(is_FC(is_NCpi0BDT(is_generic(df_badmatch)))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).reco_pi0_costheta]
weight_int = [is_Np(is_FC(is_NCpi0BDT(is_generic(df_nueCCinFV)))).weight]

hist_dirt = is_Np(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).reco_pi0_costheta
weight_dirt = is_Np(is_FC(is_NCpi0BDT(is_generic(df_dirt)))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='deepskyblue', fmt='o', label=r'FC Np')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$cos\theta^{\pi^{0}}_{reco}$')
plt.ylabel('Purity')



fig.add_subplot(122)
hist = is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV))).reco_pi0_costheta
weight = is_0p(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight

hist_ext = is_0p(is_NCpi0BDT(is_generic(df_ext))).reco_pi0_costheta
weight_ext = is_0p(is_NCpi0BDT(is_generic(df_ext))).weight

hist_over = [is_0p(is_NCpi0BDT(is_generic(df_NCinFV))).reco_pi0_costheta,
             is_0p(is_NCpi0BDT(is_generic(df_outFV))).reco_pi0_costheta,
             is_0p(is_NCpi0BDT(is_generic(df_CCpi0inFV))).reco_pi0_costheta,
             is_0p(is_NCpi0BDT(is_generic(df_numuCCinFV))).reco_pi0_costheta,
             is_0p(is_NCpi0BDT(is_generic(df_badmatch))).reco_pi0_costheta]
weight_over = [is_0p(is_NCpi0BDT(is_generic(df_NCinFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_outFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_CCpi0inFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_numuCCinFV))).weight,
               is_0p(is_NCpi0BDT(is_generic(df_badmatch))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_0p(is_NCpi0BDT(is_generic(df_nueCCinFV))).reco_pi0_costheta]
weight_int = [is_0p(is_NCpi0BDT(is_generic(df_nueCCinFV))).weight]

hist_dirt = is_0p(is_NCpi0BDT(is_generic(df_dirt))).reco_pi0_costheta
weight_dirt = is_0p(is_NCpi0BDT(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='lightcoral', fmt='o', label=r'FC+PC 0p')
# -------------------------------------------------------------------------------------------------------------------
hist = is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV))).reco_pi0_costheta
weight = is_Np(is_NCpi0BDT(is_generic(df_NCpi0inFV))).weight

hist_ext = is_Np(is_NCpi0BDT(is_generic(df_ext))).reco_pi0_costheta
weight_ext = is_Np(is_NCpi0BDT(is_generic(df_ext))).weight

hist_over = [is_Np(is_NCpi0BDT(is_generic(df_NCinFV))).reco_pi0_costheta,
             is_Np(is_NCpi0BDT(is_generic(df_outFV))).reco_pi0_costheta,
             is_Np(is_NCpi0BDT(is_generic(df_CCpi0inFV))).reco_pi0_costheta,
             is_Np(is_NCpi0BDT(is_generic(df_numuCCinFV))).reco_pi0_costheta,
             is_Np(is_NCpi0BDT(is_generic(df_badmatch))).reco_pi0_costheta]
weight_over = [is_Np(is_NCpi0BDT(is_generic(df_NCinFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_outFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_CCpi0inFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_numuCCinFV))).weight,
               is_Np(is_NCpi0BDT(is_generic(df_badmatch))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_Np(is_NCpi0BDT(is_generic(df_nueCCinFV))).reco_pi0_costheta]
weight_int = [is_Np(is_NCpi0BDT(is_generic(df_nueCCinFV))).weight]

hist_dirt = is_Np(is_NCpi0BDT(is_generic(df_dirt))).reco_pi0_costheta
weight_dirt = is_Np(is_NCpi0BDT(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='deepskyblue', fmt='o', label=r'FC+PC Np')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})
plt.xlabel(r'$cos\theta^{\pi^{0}}_{reco}$')
plt.tight_layout()
plt.savefig('plot_open_dataset/pur_reco_pi0_costheta_Xp.png')
plt.show()

## Investigation

In [ ]:
math.log10((1.+np.tanh(logit(1e-5)))/(1.-np.tanh(logit(1e-5))))

In [ ]:
math.log10((1.+np.tanh(logit(0.9999)))/(1.-np.tanh(logit(0.9999))))

In [ ]:
print('BNB data: %i'%(df_data[df_data.nc_pio_score < -9.5].shape[0]))
print('Cosmic: %i'%(df_badmatch[df_badmatch.nc_pio_score < -9.5].shape[0]))
print('EXT: %i'%(df_ext[df_ext.nc_pio_score < -9.5].shape[0]))
print('DIRT: %i'%(df_dirt[df_dirt.nc_pio_score < -9.5].shape[0]))
print('out FV: %i'%(df_outFV[df_outFV.nc_pio_score < -9.5].shape[0]))
print('NCpi0 FV: %i'%(df_NCpi0inFV[df_NCpi0inFV.nc_pio_score < -9.5].shape[0]))
print('NC FV: %i'%(df_NCinFV[df_NCinFV.nc_pio_score < -9.5].shape[0]))
print('CCpi0 FV: %i'%(df_CCpi0inFV[df_CCpi0inFV.nc_pio_score < -9.5].shape[0]))
print('CC FV: %i'%(df_numuCCinFV[df_numuCCinFV.nc_pio_score < -9.5].shape[0]))
print('nue FV: %i'%(df_nueCCinFV[df_nueCCinFV.nc_pio_score < -9.5].shape[0]))

In [ ]:
print('BNB data: %i'%(df_data[df_data.nc_pio_score > 7.5].shape[0]))
print('Cosmic: %i'%(df_badmatch[df_badmatch.nc_pio_score > 7.5].shape[0]))
print('EXT: %i'%(df_ext[df_ext.nc_pio_score > 7.5].shape[0]))
print('DIRT: %i'%(df_dirt[df_dirt.nc_pio_score > 7.5].shape[0]))
print('out FV: %i'%(df_outFV[df_outFV.nc_pio_score > 7.5].shape[0]))
print('NCpi0 FV: %i'%(df_NCpi0inFV[df_NCpi0inFV.nc_pio_score > 7.5].shape[0]))
print('NC FV: %i'%(df_NCinFV[df_NCinFV.nc_pio_score > 7.5].shape[0]))
print('CCpi0 FV: %i'%(df_CCpi0inFV[df_CCpi0inFV.nc_pio_score > 7.5].shape[0]))
print('CC FV: %i'%(df_numuCCinFV[df_numuCCinFV.nc_pio_score > 7.5].shape[0]))
print('nue FV: %i'%(df_nueCCinFV[df_nueCCinFV.nc_pio_score > 7.5].shape[0]))

In [ ]:
data = is_NCpi0BDT(is_generic(df_NCpi0inFV))[is_NCpi0BDT(is_generic(df_NCpi0inFV)).truth_nuEnergy < 500]
BINS = [0,50,100,150,200,300,400,500]
MAX = BINS[-1]

range_X = [0,MAX]
range_Y = [0,MAX]
weight_XY = data.weight
label_X = r'Truth energy [MeV]'
label_Y = r'Reco energy [MeV]'
hist_X = data.truth_pio_energy_1*1000.
hist_Y = data.kine_pio_energy_1

plt.figure(figsize=(14,5))
plt.suptitle(r'Nu Energy < 600 MeV')
plt.subplot(121)
plt.title('First shower')
plt.hist2d(x=hist_X, y=hist_Y, bins=[BINS,BINS], range=[range_X,range_Y], cmap='PuRd')
x = np.linspace(0,MAX,MAX)
plt.plot(x,x,c='red')
plt.xlabel(label_X)
plt.ylabel(label_Y)
plt.colorbar()
plt.subplot(122)
plt.title('Second shower')
hist_X = data.truth_pio_energy_2*1000.
hist_Y = data.kine_pio_energy_2
plt.hist2d(x=hist_X, y=hist_Y, bins=[BINS,BINS], range=[range_X,range_Y], cmap='PuRd')
x = np.linspace(0,MAX,MAX)
plt.plot(x,x,c='red')
plt.xlabel(label_X)
plt.ylabel(label_Y)
plt.colorbar()

plt.tight_layout()
#plt.savefig('plot_open_dataset/NCpi0BDT_TvR_pi0_energy_1.png')
plt.show()

# Might be Useful, Might not Be

### two_plot panels

In [ ]:
hist_energy = [x[x.kine_reco_Enu < 2000].kine_reco_Enu for x in samples]
w_ = [x[x.kine_reco_Enu < 2000].weight for x in samples]
l_ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]

hist_pi0mass = [x[(x.kine_pio_mass < 300) & (x.kine_pio_mass > 10)].kine_pio_mass for x in samples]
w__ = [x[(x.kine_pio_mass < 300) & (x.kine_pio_mass > 10)].weight for x in samples]
l__ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w__)]

two_panel_plot([hist_energy,hist_pi0mass], [w_,w__], [20,29], [(0,2000),(10,300)], [(0,1500),(0,800)], [colors,colors], [l_,l__], 
               [r'Reco $\nu$ energy [MeV]',r'Reco $\pi^{0}$ mass [MeV/c$^{2}$]'], ['Event count','Event count'])

# Evaluate DATA/Prediction
print('DATA/Prediction (left): %1.3f'%(sum(w_[0]) / sum([sum(x) for x in w_[1:]])))
print('DATA/Prediction (right): %1.3f'%(sum(w__[0]) / sum([sum(x) for x in w__[1:]])))

In [ ]:
hist_pi0energy = [x[x.reco_pi0_energy < 1000].reco_pi0_energy for x in samples]
w_ = [x[x.reco_pi0_energy < 1000].weight for x in samples]
l_ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]

hist_pi0momentum = [x[x.reco_pi0_momentum < 1000].reco_pi0_momentum for x in samples]
w__ = [x[x.reco_pi0_momentum < 1000].weight for x in samples]
l__ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w__)]

two_panel_plot([hist_pi0energy,hist_pi0momentum], [w_,w__], [10,10], [(0,1000),(0,1000)], [(0,3700),(0,2500)], [colors,colors], [l_,l__], 
               [r'Reco $\pi^{0}$ energy [MeV]',r'Reco $\pi^{0}$ momentum [MeV/c]'], ['Event count','Event count'])

# Evaluate DATA/Prediction
print('DATA/Prediction (left): %1.3f'%(sum(w_[0]) / sum([sum(x) for x in w_[1:]])))
print('DATA/Prediction (right): %1.3f'%(sum(w__[0]) / sum([sum(x) for x in w__[1:]])))

### Effiency, Purity, Combined plots

In [ ]:
bins = np.linspace(0.,2500.,26)
xrange = (0,2500)

fig = plt.figure(figsize=(8,5.5))
ax = fig.add_subplot(111)
return_eff_plot(is_FC(NCpi0(is_generic(df_NC_pi0))).truth_nuEnergy,
                is_FC(NCpi0(is_generic(df_NC_pi0))).weight,
                list(is_PC(NCpi0(is_generic(Active(df_NC_pi0_all)))).truth_nuEnergy)+list(not_NCpi0(is_generic(Active(df_NC_pi0_all))).truth_nuEnergy)+list(is_not_generic(Active(df_NC_pi0_all)).truth_nuEnergy),
                list(is_PC(NCpi0(is_generic(Active(df_NC_pi0_all)))).weight)+list(not_NCpi0(is_generic(Active(df_NC_pi0_all))).weight)+list(is_not_generic(Active(df_NC_pi0_all)).weight),
                bins, xrange, color='lightcoral', fmt='o', label=r'FC NC $\pi^{0}$ (Efficiency)')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(NCpi0(is_generic(df_NC_pi0)).truth_nuEnergy,
                NCpi0(is_generic(df_NC_pi0)).weight,
                list(not_NCpi0(is_generic(Active(df_NC_pi0_all))).truth_nuEnergy)+list(is_not_generic(Active(df_NC_pi0_all)).truth_nuEnergy),
                list(not_NCpi0(is_generic(Active(df_NC_pi0_all))).weight)+list(is_not_generic(Active(df_NC_pi0_all)).weight),
                bins, xrange, color='red', fmt='o', label=r'FC+PC NC $\pi^{0}$ (Efficiency)')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$E^{\nu}_{true}$ [MeV]')
ax.set_ylabel('Efficiency')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/efficiency_trueEnergy.png')
plt.show()

In [ ]:
bins = np.linspace(0.,2500.,26)
xrange = (0,2500)

fig = plt.figure(figsize=(8,5.5))
ax = fig.add_subplot(111)
hist = is_FC(NCpi0(is_generic(df_NC_pi0))).kine_reco_Enu
weight = is_FC(NCpi0(is_generic(df_NC_pi0))).weight

hist_ext = is_FC(NCpi0(is_generic(df_ext))).kine_reco_Enu
weight_ext = is_FC(NCpi0(is_generic(df_ext))).weight

hist_over = [is_FC(NCpi0(is_generic(df_NC_))).kine_reco_Enu,
             is_FC(NCpi0(is_generic(df_outFV))).kine_reco_Enu,
             is_FC(NCpi0(is_generic(df_CC_pi0))).kine_reco_Enu,
             is_FC(NCpi0(is_generic(df_CC_))).kine_reco_Enu,
             is_FC(NCpi0(is_generic(df_cosmic))).kine_reco_Enu]
weight_over = [is_FC(NCpi0(is_generic(df_NC_))).weight,
               is_FC(NCpi0(is_generic(df_outFV))).weight,
               is_FC(NCpi0(is_generic(df_CC_pi0))).weight,
               is_FC(NCpi0(is_generic(df_CC_))).weight,
               is_FC(NCpi0(is_generic(df_cosmic))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_FC(NCpi0(is_generic(df_nue))).kine_reco_Enu]
weight_int = [is_FC(NCpi0(is_generic(df_nue))).weight]

hist_dirt = is_FC(NCpi0(is_generic(df_dirt))).kine_reco_Enu
weight_dirt = is_FC(NCpi0(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='deepskyblue', fmt='o', label=r'FC NC $\pi^{0}$(Purity)')
# -------------------------------------------------------------------------------------------------------------------
hist = NCpi0(is_generic(df_NC_pi0)).kine_reco_Enu
weight = NCpi0(is_generic(df_NC_pi0)).weight

hist_ext = NCpi0(is_generic(df_ext)).kine_reco_Enu
weight_ext = NCpi0(is_generic(df_ext)).weight

hist_over = [NCpi0(is_generic(df_NC_)).kine_reco_Enu,
             NCpi0(is_generic(df_outFV)).kine_reco_Enu,
             NCpi0(is_generic(df_CC_pi0)).kine_reco_Enu,
             NCpi0(is_generic(df_CC_)).kine_reco_Enu,
             NCpi0(is_generic(df_cosmic)).kine_reco_Enu]
weight_over = [NCpi0(is_generic(df_NC_)).weight,
               NCpi0(is_generic(df_outFV)).weight,
               NCpi0(is_generic(df_CC_pi0)).weight,
               NCpi0(is_generic(df_CC_)).weight,
               NCpi0(is_generic(df_cosmic)).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [NCpi0(is_generic(df_nue)).kine_reco_Enu]
weight_int = [NCpi0(is_generic(df_nue)).weight]

hist_dirt = NCpi0(is_generic(df_dirt)).kine_reco_Enu
weight_dirt = NCpi0(is_generic(df_dirt)).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='blue', fmt='o', label=r'FC+PC NC $\pi^{0}$(Purity)')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

ax.set_xlabel(r'$E^{\nu}_{reco}$ [MeV]')
ax.set_ylabel('Purity')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')
plt.tight_layout()
plt.savefig('plot_open_dataset/purity_recoEnergy.png')
plt.show()

In [ ]:
bins = np.linspace(0.,2500.,26)
xrange = (0,2500)

fig = plt.figure(figsize=(8,5.5))
ax = fig.add_subplot(111)
return_eff_plot(is_FC(NCpi0(is_generic(df_NC_pi0))).truth_nuEnergy,
                is_FC(NCpi0(is_generic(df_NC_pi0))).weight,
                list(is_PC(NCpi0(is_generic(Active(df_NC_pi0_all)))).truth_nuEnergy)+list(not_NCpi0(is_generic(Active(df_NC_pi0_all))).truth_nuEnergy)+list(is_not_generic(Active(df_NC_pi0_all)).truth_nuEnergy),
                list(is_PC(NCpi0(is_generic(Active(df_NC_pi0_all)))).weight)+list(not_NCpi0(is_generic(Active(df_NC_pi0_all))).weight)+list(is_not_generic(Active(df_NC_pi0_all)).weight),
                bins, xrange, color='lightcoral', fmt='o', label=r'FC NC $\pi^{0}$ (Efficiency)')
# -------------------------------------------------------------------------------------------------------------------
return_eff_plot(NCpi0(is_generic(df_NC_pi0)).truth_nuEnergy,
                NCpi0(is_generic(df_NC_pi0)).weight,
                list(not_NCpi0(is_generic(Active(df_NC_pi0_all))).truth_nuEnergy)+list(is_not_generic(Active(df_NC_pi0_all)).truth_nuEnergy),
                list(not_NCpi0(is_generic(Active(df_NC_pi0_all))).weight)+list(is_not_generic(Active(df_NC_pi0_all)).weight),
                bins, xrange, color='red', fmt='o', label=r'FC+PC NC $\pi^{0}$ (Efficiency)')
# -------------------------------------------------------------------------------------------------------------------
hist = is_FC(NCpi0(is_generic(df_NC_pi0))).kine_reco_Enu
weight = is_FC(NCpi0(is_generic(df_NC_pi0))).weight

hist_ext = is_FC(NCpi0(is_generic(df_ext))).kine_reco_Enu
weight_ext = is_FC(NCpi0(is_generic(df_ext))).weight

hist_over = [is_FC(NCpi0(is_generic(df_NC_))).kine_reco_Enu,
             is_FC(NCpi0(is_generic(df_outFV))).kine_reco_Enu,
             is_FC(NCpi0(is_generic(df_CC_pi0))).kine_reco_Enu,
             is_FC(NCpi0(is_generic(df_CC_))).kine_reco_Enu,
             is_FC(NCpi0(is_generic(df_cosmic))).kine_reco_Enu]
weight_over = [is_FC(NCpi0(is_generic(df_NC_))).weight,
               is_FC(NCpi0(is_generic(df_outFV))).weight,
               is_FC(NCpi0(is_generic(df_CC_pi0))).weight,
               is_FC(NCpi0(is_generic(df_CC_))).weight,
               is_FC(NCpi0(is_generic(df_cosmic))).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [is_FC(NCpi0(is_generic(df_nue))).kine_reco_Enu]
weight_int = [is_FC(NCpi0(is_generic(df_nue))).weight]

hist_dirt = is_FC(NCpi0(is_generic(df_dirt))).kine_reco_Enu
weight_dirt = is_FC(NCpi0(is_generic(df_dirt))).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='deepskyblue', fmt='o', label=r'FC NC $\pi^{0}$(Purity)')
# -------------------------------------------------------------------------------------------------------------------
hist = NCpi0(is_generic(df_NC_pi0)).kine_reco_Enu
weight = NCpi0(is_generic(df_NC_pi0)).weight

hist_ext = NCpi0(is_generic(df_ext)).kine_reco_Enu
weight_ext = NCpi0(is_generic(df_ext)).weight

hist_over = [NCpi0(is_generic(df_NC_)).kine_reco_Enu,
             NCpi0(is_generic(df_outFV)).kine_reco_Enu,
             NCpi0(is_generic(df_CC_pi0)).kine_reco_Enu,
             NCpi0(is_generic(df_CC_)).kine_reco_Enu,
             NCpi0(is_generic(df_cosmic)).kine_reco_Enu]
weight_over = [NCpi0(is_generic(df_NC_)).weight,
               NCpi0(is_generic(df_outFV)).weight,
               NCpi0(is_generic(df_CC_pi0)).weight,
               NCpi0(is_generic(df_CC_)).weight,
               NCpi0(is_generic(df_cosmic)).weight]
cum = []
for sublist in hist_over:
    for item in sublist: cum.append(item)        
weight_cum = []
for sublist in weight_over:
    for item in sublist: weight_cum.append(item)
        
hist_int = [NCpi0(is_generic(df_nue)).kine_reco_Enu]
weight_int = [NCpi0(is_generic(df_nue)).weight]

hist_dirt = NCpi0(is_generic(df_dirt)).kine_reco_Enu
weight_dirt = NCpi0(is_generic(df_dirt)).weight

back_hists = [hist_ext, cum, hist_int, hist_dirt]
back_weights = [weight_ext, weight_cum, weight_int, weight_dirt]

sig_scale = W_over
back_scales = [W_ext, W_over, W_int, W_dirt]

return_purity_plot(hist, weight, back_hists, back_weights, bins, xrange, color='blue', fmt='o', label=r'FC+PC NC $\pi^{0}$(Purity)')
# -------------------------------------------------------------------------------------------------------------------
plt.xlim(xrange)
plt.ylim(0,1)
#plt.xlabel(r'$E^{\nu}_{true}$ [MeV]')
#plt.ylabel('Efficiency')
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(loc='best', prop={'size': legend_size})

plt.twiny()
plt.xlim(xrange)
plt.ylim(0,1)
plt.xlabel(r'$E^{\nu}_{reco}$ [MeV]')
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)

plt.twinx()
plt.ylabel('Purity')
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)

ax.set_xlabel(r'$E^{\nu}_{true}$ [MeV]')
ax.set_ylabel('Efficiency')
ax.spines['left'].set_color('red')
ax.spines['bottom'].set_color('red')
ax.xaxis.label.set_color('red')
ax.yaxis.label.set_color('red')
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')

plt.savefig('plot_open_dataset/efficiency_purity_combined.png')
plt.show()

# Cut-based Internal note (current Wire-Cell NCpi0 cut)

### FC+PC (Plots)

In [ ]:
colors =   ['black',  'brown',    'paleturquoise',       'orange',          'purple',  'saddlebrown', 'gold',    'limegreen',            'aqua',           'springgreen']
labels =   ['Data',   'Cosmic',   r'NC $\pi^{0}$ in FV', 'NC other in FV',  'Dirt',    'EXT',         'outFV',   r'CC $\pi^{0}$ in FV',  'CC other in FV', r'$\nu_{e}$CC in FV']
families = ['DATA',   'MC',       'MC',                  'MC',              'MC',      'DATA',        'MC',      'MC',                   'MC',             'MC']
samples =  [df_data,  df_cosmic,  df_NC_pi0,             df_NC_,            df_dirt,   df_ext,        df_outFV,  df_CC_pi0,              df_CC_,           df_nue]
samples =  [is_pi0(is_NC(is_generic(x)), family=y) for x,y in zip(samples,families)]

# -------------------------------------------------------------------------

num = sum(is_pi0(is_NC(is_generic(df_NC_pi0)), family='MC').weight)
den = sum(Active(df_NC_pi0_all).weight)
print('Total efficiency: %1.3f / %1.3f = %1.3f'%(num, den, num/den))

w = [x.weight for x in samples]    # Doesn't depend on range of plots
signal = sum(w[2])                 # df_NC_pi0
tot = sum([sum(x) for x in w[1:]]) # Excluding data
print('Total purity: %1.3f / %1.3f = %1.3f'%(signal, tot, signal/tot))

In [ ]:
hist_energy = [x[x.kine_reco_Enu < 2000].kine_reco_Enu for x in samples]
w_ = [x[x.kine_reco_Enu < 2000].weight for x in samples]
l_ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]

hist_pi0mass = [x[x.kine_pio_mass < 300].kine_pio_mass for x in samples]
w__ = [x[x.kine_pio_mass < 300].weight for x in samples]
l__ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w__)]

two_panel_plot([hist_energy,hist_pi0mass], [w_,w__], [20,30], [(0,2000),(0,300)], [(0,200),(0,74)], [colors,colors], [l_,l__], 
               [r'Reco $\nu$ energy [MeV]',r'Reco $\pi^{0}$ mass [MeV/c$^{2}$]'], ['Event count','Event count'])

# Evaluate DATA/Prediction
print('DATA/Prediction (left): %1.3f'%(sum(w_[0]) / sum([sum(x) for x in w_[1:]])))
print('DATA/Prediction (right): %1.3f'%(sum(w__[0]) / sum([sum(x) for x in w__[1:]])))

In [ ]:
hist_pi0energy = [x[x.reco_pi0_energy < 1000].reco_pi0_energy for x in samples]
w_ = [x[x.reco_pi0_energy < 1000].weight for x in samples]
l_ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]

hist_pi0momentum = [x[x.reco_pi0_momentum < 1000].reco_pi0_momentum for x in samples]
w__ = [x[x.reco_pi0_momentum < 1000].weight for x in samples]
l__ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w__)]

two_panel_plot([hist_pi0energy,hist_pi0momentum], [w_,w__], [10,10], [(0,1000),(0,1000)], [(0,340),(0,300)], [colors,colors], [l_,l__], 
               [r'Reco $\pi^{0}$ energy [MeV]',r'Reco $\pi^{0}$ momentum [MeV/c]'], ['Event count','Event count'])

# Evaluate DATA/Prediction
print('DATA/Prediction (left): %1.3f'%(sum(w_[0]) / sum([sum(x) for x in w_[1:]])))
print('DATA/Prediction (right): %1.3f'%(sum(w__[0]) / sum([sum(x) for x in w__[1:]])))

### FC (Plots)

In [ ]:
colors =   ['black',  'brown',    'paleturquoise',       'orange',          'purple',  'saddlebrown', 'gold',    'limegreen',            'aqua',           'springgreen']
labels =   ['Data',   'Cosmic',   r'NC $\pi^{0}$ in FV', 'NC other in FV',  'Dirt',    'EXT',         'outFV',   r'CC $\pi^{0}$ in FV',  'CC other in FV', r'$\nu_{e}$CC in FV']
families = ['DATA',   'MC',       'MC',                  'MC',              'MC',      'DATA',        'MC',      'MC',                   'MC',             'MC']
samples =  [df_data,  df_cosmic,  df_NC_pi0,             df_NC_,            df_dirt,   df_ext,        df_outFV,  df_CC_pi0,              df_CC_,           df_nue]
samples =  [is_FC(is_pi0(is_NC(is_generic(x)), family=y)) for x,y in zip(samples,families)]

# -------------------------------------------------------------------------

num = sum(is_FC(is_pi0(is_NC(is_generic(df_NC_pi0)), family='MC')).weight)
den = sum(Active(df_NC_pi0_all).weight)
print('Total efficiency: %1.3f / %1.3f = %1.3f'%(num, den, num/den))

w = [x.weight for x in samples]    # Doesn't depend on range of plots
signal = sum(w[2])                 # df_NC_pi0
tot = sum([sum(x) for x in w[1:]]) # Excluding data
print('Total purity: %1.3f / %1.3f = %1.3f'%(signal, tot, signal/tot))

In [ ]:
hist_energy = [x[x.kine_reco_Enu < 2000].kine_reco_Enu for x in samples]
w_ = [x[x.kine_reco_Enu < 2000].weight for x in samples]
l_ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]

hist_pi0mass = [x[x.kine_pio_mass < 300].kine_pio_mass for x in samples]
w__ = [x[x.kine_pio_mass < 300].weight for x in samples]
l__ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w__)]

two_panel_plot([hist_energy,hist_pi0mass], [w_,w__], [20,30], [(0,2000),(0,300)], [(0,200),(0,74)], [colors,colors], [l_,l__], 
               [r'Reco $\nu$ energy [MeV]',r'Reco $\pi^{0}$ mass [MeV/c$^{2}$]'], ['Event count','Event count'])

# Evaluate DATA/Prediction
print('DATA/Prediction (left): %1.3f'%(sum(w_[0]) / sum([sum(x) for x in w_[1:]])))
print('DATA/Prediction (right): %1.3f'%(sum(w__[0]) / sum([sum(x) for x in w__[1:]])))

In [ ]:
hist_pi0energy = [x[x.reco_pi0_energy < 1000].reco_pi0_energy for x in samples]
w_ = [x[x.reco_pi0_energy < 1000].weight for x in samples]
l_ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w_)]

hist_pi0momentum = [x[x.reco_pi0_momentum < 1000].reco_pi0_momentum for x in samples]
w__ = [x[x.reco_pi0_momentum < 1000].weight for x in samples]
l__ = [x+' (%1.1f)'%(sum(y)) for x,y in zip(labels,w__)]

two_panel_plot([hist_pi0energy,hist_pi0momentum], [w_,w__], [10,10], [(0,1000),(0,1000)], [(0,340),(0,300)], [colors,colors], [l_,l__], 
               [r'Reco $\pi^{0}$ energy [MeV]',r'Reco $\pi^{0}$ momentum [MeV/c]'], ['Event count','Event count'])

# Evaluate DATA/Prediction
print('DATA/Prediction (left): %1.3f'%(sum(w_[0]) / sum([sum(x) for x in w_[1:]])))
print('DATA/Prediction (right): %1.3f'%(sum(w__[0]) / sum([sum(x) for x in w__[1:]])))